# Österreich: Vermischtes Monitoring zu Gesundheits- & SARS-CoV-2-Daten

Erstellt aus <https://github.com/zeitferne/covidat-tools/blob/main/notebooks/monitoring.ipynb>

Alle Darstellungen in diesem Dokument basieren zwar im Grunde auf Daten aus offiziellen Quellen, enthalten aber meist zusätzliche Berechnungen oder sonstige Verarbeitungsschritte und können daher in keiner Weise als offiziell betrachtet werden. Fehler sind möglich und wahrscheinlich, alle Angaben ohne Gewähr. Dieses Dokument ist ein Hobbyprojekt und nicht für eine breite Öffentlichkeit gedacht. Auf Anfragen zu bestimmten Daten auf Social Media reagiere ich jedoch wahrscheinlich, wenn auch möglicherweise nicht sehr zeitnah.

Für weitere Angaben, siehe Ende des Dokuments.

Inhalt:

* [SARS-CoV-2-Abwassermonitoring](#SARS-CoV-2-Abwassermonitoring)
   * [Varianten](#Varianten)
* [Medikamentenmangel](#Medikamentenmangel)
* [SARI-Krankenhaus-Monitoring](#SARI-Krankenhaus-Monitoring)
* [Krankenstand](#Krankenstand)
* [Sterberaten](#Sterberaten)
* [Influenza und Atemwegserkrankungen](#Influenza-und-Atemwegserkrankungen) (erst ca. ab Oktober wieder aktualisiert)

In [ ]:
# @export: --no-input

In [ ]:
import locale

locale.setlocale(locale.LC_ALL, "de_AT.UTF-8")

import os

os.environ["TZ"] = "Europe/Vienna"
import time

if hasattr(time, "tzset"):
    time.tzset()

In [ ]:
%matplotlib inline
%precision %.4f
# %load_ext snakeviz
import colorcet
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates
import matplotlib.ticker
import matplotlib.colors
import seaborn as sns
from importlib import reload
import collections
from datetime import date, datetime, timedelta, timezone, UTC
from IPython.display import display, Markdown, HTML
import re
from pathlib import Path
from itertools import chain
import json
import warnings

from covidat import util, covdata, cov, collectshortage

pd.options.display.precision = 4

plt.rcParams['figure.figsize'] = (16 * 0.7, 9 * 0.7)
plt.rcParams['figure.dpi'] = 120  # 80
plt.rcParams['figure.facecolor'] = '#fff'
sns.set_theme(style="whitegrid")
sns.set_palette('tab10')

plt.rcParams['image.cmap'] = cov.un_cmap

pd.options.display.max_rows = 120
pd.options.display.min_rows = 40

In [ ]:
plt_experimental = bool(os.environ.get("COVAT_NB_EXPERIMENTAL"))

In [ ]:
statbev = cov.load_bev()
pandembev = cov.bev_to_daily(statbev)
pandembev_at = pandembev.xs(10, level="BundeslandID").groupby("Datum").sum()

In [ ]:
display(
    Markdown(
        f"Zuletzt erstellt am *{datetime.now(UTC).astimezone().strftime('%a %d.%m.%Y, %H:%M Uhr %Z')}* (Daten können älter sein, siehe einzelne Diagramme)"
    )
)

In [ ]:
if plt_experimental:
    fig, ax = plt.subplots(layout="constrained")
    pandembev_at.plot()
    fig.suptitle("Österreich: Grunddaten: Bevölkerungsentwicklung, lt. Statistik Austria Hauptszenario")


def calc1ydate(ycol, dasr):
    # Align around Gregorian year 2022. Most ISO-week-years start slightly before the Gregorian year,
    # but the ISO year of the Sunday (7th day) of the first ISO week will match the Gregorian year.
    return pd.Timestamp(year=2022, month=1, day=1) + (
        dasr["date"]
        - dasr[ycol].map(lambda y: pd.Timestamp(year=pd.Timestamp.fromisocalendar(y, 1, 7).year, month=1, day=1))
    )

# SARS-CoV-2-Abwassermonitoring

* Daten von: Medizinische Universität Innsbruck (Durchführung), hydro-IT GmbH, AGES (IT, Statistik), Bundesministerium für Soziales, Gesundheit, Pflege und Konsumentenschutz (BMSGPK) (Auftrag)
* Daten-URL: <https://abwassermonitoring.at/dashboard/>
* Daten-Aktualisierung: Mehrmals wöchentlich, üblicherweise täglich von Dienstag bis inkl. Samstag
  (Dienstags meist nur minimale Korrekturen)

In [ ]:
blverlauf = covdata.load_ww_blverlauf()
blverlauf["FileDate"] = blverlauf["FileDate"].dt.tz_convert(util.TZ_AT).dt.normalize()
blverlauf.drop_duplicates(["FileDate", "Datum", "Bundesland"], keep="last", inplace=True)  # Normalize intra-day
blvfull = blverlauf[blverlauf["Datum"] >= pd.to_datetime("2022-09-01")]
abwfrom = blverlauf["FileDate"].min()
blverlauf_last = blverlauf[blverlauf["FileDate"] == blverlauf["FileDate"].max()]
blv1 = covdata.first_filedate(blvfull, ["Bundesland"])


def abw_tinfo(ts):
    return (
        f"Darstellung: @zeitferne | Daten: abwassermonitoring.at/dashboard, Stand"
        f" {ts['FileDate'].max().tz_convert('Europe/Vienna').strftime('%d.%m.%y')}"
    )


DS_WW = "abwassermonitoring.at (MedUni Innsbruck, hydro-IT, AGES, " + cov.DS_BMG + ")"
DS_WW_VAR = "abwassermonitoring.at (CeMM, ÖAW, MedUni Innsbruck, hydro-IT, AGES, " + cov.DS_BMG + ")"
ABW_TINFO = abw_tinfo(blverlauf_last)

In [ ]:
rww = pd.read_csv(
    util.COLLECTROOT / "covid/abwassermonitoring/natmon_02_all.csv.xz", sep=";", parse_dates=["FileDate", "Datum"]
).pivot_table(index=["FileDate", "Datum"], columns=["typ"], values="y")

In [ ]:
natmon_latest = (
    pd.read_csv(
        util.DATAROOT / "covid/abwassermonitoring/dl_natmon_01_latest.csv",
        sep=";",
        encoding="utf-8",
        parse_dates=["Datum"],
    )
    .rename(
        errors="raise",
        columns={
            "gemittelte kumulierte Faelle der letzten 7 Tage aus den Humantestungen": "AnzahlFaelle7Tage",
            "Gesamtvirenfracht [Genkopien pro Tag * 10^6]": "Abwassersignal",
        },
    )
    .set_index("Datum")
)

natmon_norm = natmon_latest["Abwassersignal"].copy()
_msk_erw = natmon_norm.index >= pd.to_datetime("2023-02-01")

# "Rund 52%" https://web.archive.org/web/20230110152232/https://abwassermonitoring.at/dashboard/
# Use upper bound because it still seems much
natmon_norm.loc[~_msk_erw] /= pandembev_at * 0.524

# "Mehr als 58%" https://abwassermonitoring.at/dashboard/
natmon_norm.loc[_msk_erw] /= pandembev_at * 0.582
natmon_norm_interp = natmon_norm.dropna().resample("1D").interpolate()
natmon_norm_sm = (
    natmon_norm.dropna()
    .pipe(lambda nn: nn.ewm(times=nn.index, halflife=timedelta(5)))
    .mean()
    .dropna()
    .resample("1D")
    .interpolate()
    .shift(-4)
)
NATMON_STAND = (
    util.fdate_from_fname((util.DATAROOT / "covid/abwassermonitoring/dl_natmon_01_latest.csv").readlink())
    .astimezone(util.TZ_AT)
    .strftime("Stand %d.%m.%y")
)
ww_completeuntil = blverlauf_last.groupby("Bundesland")["Datum"].max().min()
ww_bl_maxdate = blverlauf_last.groupby("Bundesland")["Datum"].max()
ww_bl_incompl = ", ".join(
    sorted(ww_bl_maxdate[ww_bl_maxdate < ww_bl_maxdate["Österreich"]].index.map(covdata.SHORTNAME_BY_BUNDESLAND))
)
ww_bl_incompl_nn = ", ".join(
    sorted(ww_bl_maxdate[ww_bl_maxdate < natmon_norm.last_valid_index()].index.map(covdata.SHORTNAME_BY_BUNDESLAND))
)
display(
    Markdown(
        "* Abwasserdaten von allen Bundesländern bis: *"
        + ww_completeuntil.strftime("%a %d.%m.%y")
        + "*\n"
        + "* Abwasserdaten von Gesamt-Ö eingetragen bis: *"
        + ww_bl_maxdate["Österreich"].strftime("%a %d.%m.%y")
        + "*"
        + ("" if not ww_bl_incompl else ", darin am Ende fehlend: " + ww_bl_incompl)
        + "\n* Ungeglätte Abwasserdaten von Gesamt-Ö eingetragen bis: *"
        + natmon_norm.last_valid_index().strftime("%a %d.%m.%y")
        + "*"
        + ("" if not ww_bl_incompl_nn else ", darin am Ende fehlend lt. geglätteten Daten: " + ww_bl_incompl_nn)
    )
)

In [ ]:
def draw_bl_ww_range(selbl, ax, logscale=True, fromdate=None, showcases=False):
    blvfull_bl = blvfull[blvfull["Bundesland"] == selbl]
    if fromdate:
        blvfull_bl = blvfull_bl[blvfull_bl["Datum"] >= fromdate]
    blvaext = (
        blvfull_bl.groupby(["Datum", "Bundesland"])
        .agg({"y": ["min", "max", "last", "first"], "FileDate": ["first", "last"]})
        .xs(selbl, level="Bundesland")
    )
    ##fig, ax = plt.subplots()
    if logscale:
        cov.set_logscale(ax)
    ax.fill_between(blvaext.index, blvaext[("y", "min")], blvaext[("y", "max")], alpha=0.3, label="Minimum‒Maximum")
    ylast = blvaext[blvaext[("FileDate", "last")] == blverlauf["FileDate"].max()][("y", "last")]
    ax.plot(ylast, color="k", label="Letzter Stand")
    ax.plot(
        blvfull_bl[blvfull_bl["FileDate"] == blvfull_bl["FileDate"].unique()[-2]].set_index("Datum")["y"],
        ls="--",
        zorder=5,
        alpha=0.8,
        lw=0.7,
        color="k",
        label="Vorheriger Stand (falls abweichend)",
    )

    # display(blvaext)
    first = True
    for _, values in blvaext.groupby(("FileDate", "first")):
        ax.plot(
            values[("y", "first")],
            marker="." if len(values) <= 1 else None,
            # markersize=,
            mew=0,
            color="C0",
            label="Erstmeldung" if first else None,
            lw=1.8,
        )
        first = False
    ax.plot(blvaext[("y", "first")], color="C0", lw=0.8)
    ax.set_facecolor("none")
    ##cov.set_date_opts(ax, blvaext.index.get_level_values("Datum"))
    ##ax.set_xlim(left=blvaext.index.get_level_values("Datum").min())
    ##ax.set_ylabel("Genkopien pro Tag / EW * 10⁶")
    ##ax.set_title(ABW_TINFO + ", Daten bis " + blvaext.index[-1].strftime("%a %d.%m.%y"), y=1.07)
    ##ax.set_xlim(*ax.get_xlim())
    if showcases:
        ax2 = ax.twinx()
        ax2.set_zorder(-3)
        if logscale:
            cov.set_logscale(ax2)
        inz = fs[(fs["Bundesland"] == selbl) & (fs["Datum"] >= blvaext.index[0])].set_index("Datum")["inz"]
        inzmax = inz.iloc[:90].max()
        ymax = ylast.iloc[:90].max()
        inzpery = inzmax / ymax
        ax2.plot(inz, color="C1", label="7-Tage-Inzidenz (rechts)" + AGES_STAMP)
        ax2.grid(False)

        def llim(x, xb):
            return x * 0.9 if logscale else x - xb

        def ulim(x, xb):
            return x * 1.1 if logscale else x + xb

        inzb = abs(ylast.min() - ylast.max()) * inzpery * 0.15

        if not logscale:
            ax2.set_ylim(bottom=0)
            ax.set_ylim(bottom=0)

        inzlims = ax2.get_ylim()
        ylims = ax.get_ylim()
        # print(selbl, inzpery, inzlims, ylims)
        inzlim_r = (min(ylims[0] * inzpery, inzlims[0]), max(ylims[1] * inzpery, inzlims[1]))
        ylim_r = (min(inzlim_r[0] / inzpery, ylims[0]), max(inzlim_r[1] / inzpery, ylims[1]))
        inzlim_r = (min(ylim_r[0] * inzpery, inzlim_r[0]), max(ylim_r[1] * inzpery, inzlim_r[1]))
        ax.set_ylim(*ylim_r)
        ax2.set_ylim(*inzlim_r)

        if False:
            ax2.set_ylim(
                min(llim(blvaext[("y", "min")].min() * inzpery, inzb), ax2.get_ylim()[0]),
                max(ulim(blvaext[("y", "max")].max() * inzpery, inzb), ax2.get_ylim()[1]),
            )
            yb = abs(inz.min() - inz.max()) / inzpery * 0.15
            ax.set_ylim(
                min(llim(inz.min() / inzpery, yb), ax.get_ylim()[0]),
                max(ulim(inz.max() / inzpery, yb), ax.get_ylim()[1]),
            )
    else:
        ax2 = None

        # ax2.set_ylim(bottom=0)
        # ax2.set_ylabel("Hospitalisierte CoV-PatientInnen")
    ##fig.legend(ncol=4, frameon=False, loc="upper center", bbox_to_anchor=(0.5, 0.95))
    # break
    # ax.set_xlim(
    #    left=fromdate,
    #    right=maxdate + max(2, timedelta(ndays * 0.05)))
    if selbl == "Österreich" and ww_completeuntil < blvaext.index.max():
        ax.axvspan(
            matplotlib.dates.date2num(ww_completeuntil),
            ax.get_xlim()[1],
            color=(1, 1, 0),
            alpha=0.2,
            label="Unvollständige Daten nach " + ww_completeuntil.strftime("%d.%m"),
        )
    return ax2, blvaext


def draw_bl_ww(selbl, logscale=True, ndays=None, ax=None, unsmooth: pd.DataFrame | None = None):
    maxdate = blvfull["Datum"].max()
    maxdays = (maxdate - blvfull["Datum"].min()).days + 1
    ndays = maxdays if ndays is None else min(ndays, maxdays)
    fromdate = maxdate - timedelta(ndays) if ndays is not None else None
    if ax is None:
        fig, ax = plt.subplots()
    else:
        fig = ax.figure
    _, blvaext = draw_bl_ww_range(selbl, ax, logscale=logscale, fromdate=fromdate)
    if unsmooth is not None:
        ax.plot(
            unsmooth[(unsmooth.index >= fromdate - timedelta(10)) & (unsmooth.index <= ww_completeuntil)].dropna(),
            label="Ungeglättet",
            color="C4",
            mew=0,
            marker=".",
        )
        ax.plot(
            unsmooth[(unsmooth.index >= fromdate - timedelta(10)) & (unsmooth.index >= ww_completeuntil)].dropna(),
            color="C4",
            mew=0,
            marker=".",
            lw=0.5,
            markersize=3,
            alpha=0.7,
        )
    cov.set_date_opts(ax, week_always=ndays < 7 * 30)
    ax.set_ylabel("Genkopien pro Tag / EW * 10⁶")
    ax.set_title(ABW_TINFO + ", Daten bis " + blvaext.index[-1].strftime("%a %d.%m.%y"), y=1.09)
    # ax.set_xlim(*ax.get_xlim())
    if logscale:
        fig.suptitle(selbl + ": SARS-CoV-2 im Abwasser, verschiedene Datenstände, logarithmische Skala", y=1.03)
    else:
        fig.suptitle(selbl + ": SARS-CoV-2 im Abwasser, verschiedene Datenstände", y=1.03)
    if ndays:
        ax.set_xlim(left=fromdate, right=maxdate + max(timedelta(2), timedelta(ndays * 0.02)))
    if not logscale:
        ax.set_ylim(0)

    if False:
        ax2 = ax.twinx()
        axlo, axhi = ax.get_ylim()
        # https://iwaponline.com/jwh/article/20/2/314/86325/Wastewater-surveillance-of-SARS-CoV-2-in-Austria
        # via https://abwassermonitoring.at/methodik/
        MCOPIES_PER_PERSON = 3e9 / 1e6
        ax2.set_ylim(axlo / MCOPIES_PER_PERSON, axhi / MCOPIES_PER_PERSON)
        ax2.set_ylabel("Bevölkerungs-Prävalenz (grob geschätzt*)")
        fig.text(
            0.95,
            0.01,
            "* Umrechnung mit 3×10⁹ Kopien/Ausscheider lt. Daleiden B. et al. 2022, https://abwassermonitoring.at/methodik/"
            "\nKann gut und gern um Faktor 3 danebenliegen, auch Varianten- & Immunstatus-abhängig unterschiedlich",
            va="bottom",
            ha="right",
            fontsize="small",
            color="grey",
        )
        if logscale:
            cov.set_logscale(ax2)
        cov.set_percent_opts(ax2, freq=0.01)
        ax2.grid(False)
    else:
        ax2 = None
    fig.legend(ncol=3, frameon=False, loc="upper center", bbox_to_anchor=(0.5, 0.97))
    # break
    return ax, ax2


def draw_each_bl_ww(logscale=True, ndays=None):
    for selbl in blverlauf["Bundesland"].unique():  # ("Österreich", "Wien", "Oberösterreich"):
        draw_bl_ww(selbl)


def draw_all_bl_ww(logscale=True, ndays=None, inz='scalefull'):
    sharey = not logscale
    fig, axs = plt.subplots(figsize=(10, 12), nrows=5, ncols=2, sharex=True, sharey=sharey)
    fig.subplots_adjust(wspace=0.3, hspace=0.17)
    maxdate = blverlauf["Datum"].max()
    fromdate = maxdate - timedelta(ndays)
    for selbl, ax in zip(
        list(cov.Bundesland.categories)[1:] + ["Österreich"], axs.flat
    ):  # ("Österreich", "Wien", "Oberösterreich"):
        ax2, blvaext = draw_bl_ww_range(selbl, ax, logscale=logscale, fromdate=fromdate if inz != 'scalefull' else None)

        title = selbl + " bis " + blvaext.index[-1].strftime("%a %d.%m.%y")
        if selbl == "Österreich":
            title += " / " + ww_completeuntil.strftime("%d.%m vollst.")
        ax.set_title(title, y=0.96)
        # ax.set_xlim(*ax.get_xlim())
        if ax is axs.flat[0]:
            fig.legend(ncol=3, frameon=False, loc="upper center", bbox_to_anchor=(0.5, 0.97), title=ABW_TINFO)
        if ax in axs[-1]:
            if ndays:
                ax.set_xlim(left=fromdate, right=maxdate + timedelta(ndays * 0.02))
            cov.set_date_opts(ax, week_always=True, showday=False)
            if ax is axs.flat[-1] and not logscale and sharey:
                ax.set_ylim(bottom=0)

        ax.grid(axis="x", which="minor", lw=0.5)
        if ax in axs.T[0]:
            ax.set_ylabel("Kopien/Tag/EW*10⁶")
        if ax2 and ax in axs.T[-1]:
            ax2.set_ylabel("Fälle/Woche/EW*10⁵")
    seplabel = "" if sharey else "separate "
    scalabel = "" if sharey and not logscale else "Skala" if sharey else "Skalen"
    if logscale:
        fig.suptitle(
            selbl + f": SARS-CoV-2 im Abwasser, verschiedene Datenstände, {seplabel}logarithmische {scalabel}", y=0.99
        )
    else:
        fig.suptitle(selbl + f": SARS-CoV-2 im Abwasser, verschiedene Datenstände, {seplabel}{scalabel}", y=0.99)

    fig.autofmt_xdate()

    # draw_all_bl_ww()


draw_bl_ww("Österreich", logscale=False, ndays=400, unsmooth=natmon_norm)

# draw_bl_ww("Wien", logscale=False, ndays=400)
draw_bl_ww("Österreich", logscale=True, ndays=180, unsmooth=natmon_norm)
draw_all_bl_ww(logscale=True, ndays=60, inz='scaledisplay')
draw_all_bl_ww(ndays=60, logscale=False, inz='scaledisplay')
# draw_each_bl_ww(logscale=True, ndays=(blvfull["Datum"].max() - pd.to_datetime("2023-02-15")).days + 1)

draw_bl_ww("Wien", logscale=False, ndays=800);

In [ ]:
if False:
    pdata = blverlauf_last  # .query("Datum >= '2022-05-15'")
    ax = sns.lineplot(
        data=pdata, x="Datum", y="coverage", hue="Bundesland", hue_order=cov.SHORTNAME_BY_BUNDESLAND.keys()
    )
    cov.set_percent_opts(ax)
    ax.legend(
        *cov.sortedlabels(ax, pdata, "coverage", fmtval=lambda v: f"{v:.0%}"),
        ncol=2,
        fontsize="x-small",
        loc="upper left",
    )
    ax.figure.suptitle("COVID-Abwassermonitoring-Abdeckung")
    ax.set_title(ABW_TINFO)
    ax.set_ylim(bottom=0)
    cov.set_date_opts(ax)
    cov.labelend2(ax, pdata, "coverage")

In [ ]:
natmon_hist = pd.read_csv(
    util.COLLECTROOT / "covid/abwassermonitoring/natmon_01_all.csv.xz", sep=";", parse_dates=["Datum", "FileDate"]
).query("name == 'Abwassersignal'")

In [ ]:
def plt_ts_mult_versions(pdata):
    pdata = pdata.copy().sort_values("FileDate", ascending=False).sort_values("Datum", kind="stable")
    bv1i = pdata.groupby("Datum").last().reset_index()[["FileDate", "Datum"]]
    # display(bv1i)
    pdi = pdata.reset_index().merge(bv1i, how="inner").set_index('index')
    pdata["sz"] = False
    pdata.loc[pdi.index, "sz"] = True
    plt.figure()
    palette = "cet_glasbey_dark"
    ax = sns.lineplot(
        data=pdata, palette=palette, x="Datum", y="y", mew=0, hue="FileDate", legend=False, lw=0.7, alpha=0.7
    )
    ax = sns.lineplot(
        data=pdata.where(pdata["sz"]),
        palette=palette,
        x="Datum",
        y="y",
        mew=0,
        hue="FileDate",
        hue_order=pdata["FileDate"].unique(),
        legend=False,
        lw=2,
        marker=".",
    )
    # display(pdata)
    title = abw_tinfo(pdata) + ", Daten bis: " + pdata.iloc[-1]["Datum"].strftime("%d.%m.%y")
    if pdata["Bundesland"].iat[0] == "Österreich" and ww_completeuntil < pdata["Datum"].max():
        ax.axvspan(
            matplotlib.dates.date2num(ww_completeuntil),
            ax.get_xlim()[1],
            color=(1, 1, 0),
            alpha=0.2,
            label="Unvollständige Daten nach " + ww_completeuntil.strftime("%d.%m"),
        )
        title += " / " + ww_completeuntil.strftime("%d.%m vollst.")
    cov.set_date_opts(ax, pdata["Datum"])
    ax.set_title(title)
    ax.set_xlim(left=pdata["Datum"].min())
    return ax

In [ ]:
def plt_natnor():
    ax = plt_ts_mult_versions(
        natmon_hist.loc[natmon_hist["Datum"] > natmon_hist["Datum"].max() - timedelta(120)]
        .dropna()
        .assign(Bundesland="Österreich")
    )
    ax.figure.suptitle(
        "Österreich: SARS-CoV-2-Abwasserlast ungegeglättet/roh, verschiedene Datenstände, logarithmische Skala"
    )
    cov.set_logscale(ax)


if plt_experimental:
    plt_natnor()

In [ ]:
for selbl in ("Österreich", "Wien"):  # blverlauf["Bundesland"].unique():
    # selbl="Österreich"
    pdata = blverlauf.query(f"Bundesland == '{selbl}'")
    pdata = pdata[pdata["Datum"] > pdata["Datum"].max() - timedelta(120)]
    ax = plt_ts_mult_versions(pdata)
    cov.set_logscale(ax)
    ax.figure.suptitle(selbl + ": SARS-CoV-2 im Abwasser, verschiedene Datenstände (logarithmisch)")
    ax.set_ylabel("Genkopien pro Tag / EW * 10⁶")

    # ax.yaxis.set_major_locator(matplotlib.ticker.AutoLocator())
    # ax.set_ylim(bottom=0)


def plt_rww(rww=rww):
    ax = plt_ts_mult_versions(
        rww[rww.index.get_level_values("Datum") > rww.index.get_level_values("Datum").max() - timedelta(120)]["rww"]
        .rename("y")
        .reset_index()
        .assign(Bundesland="Österreich")
    )
    ax.axhline(1, color="k")
    ax.set_ylabel("R eff")
    ax.set_xlim(right=ax.get_xlim()[1] + 2)
    ax.figure.suptitle("Österreich: SARS-CoV-2-Abwasser-Trendindikator, verschiedene Datenstände")


plt_rww()

In [ ]:
def plt_ww_cmp2(selbl):
    ww_oo = blverlauf.query(f"Bundesland == '{selbl}' and Datum >= '2022-09-26'")
    ww_newer = ww_oo[ww_oo["FileDate"] == ww_oo["FileDate"].max()].set_index("Datum")
    # ww_older = ww_oo.query("FileDate == '2023-09-06 23:09:35+0000'").set_index("Datum")

    fig, ax = plt.subplots(layout="constrained", dpi=300)
    # ax.plot(ww_older["y"], label="Stand " + ww_newer["FileDate"].iat[-1].tz_convert(util.TZ_AT).strftime("%a %x"))
    ax.plot(ww_newer["y"], label="Nationales Monitoring: Letzter Stand")
    ax.plot(
        covdata.first_filedate(ww_oo, ["Bundesland"]).reset_index("Bundesland")["y"],
        label="Nationales Monitoring: Zuerst angezeigte Stände, zusammengestückelt",
    )
    cov.set_date_opts(ax, week_always=True)
    ax.legend()
    fig.suptitle(f"Abwassermonitoring: {selbl}")
    ax.set_title(ABW_TINFO)
    ax.set_ylabel("Genkopien pro Tag / EW * 10⁶")


if plt_experimental:
    plt_ww_cmp2("Österreich")

In [ ]:
def plt_ww_yoy(radius=None):
    fig, ax = plt.subplots(layout="constrained")
    blvat = blverlauf_last.query("Bundesland == 'Österreich'")
    nn_old = blvat[blvat["Datum"] < natmon_norm.dropna().index[0]].set_index("Datum")["y"]

    nn = natmon_norm.dropna().combine_first(nn_old).to_frame()
    nn["jun_year"] = nn.index.to_period("A-MAY").year - 1
    tgtyear = nn["jun_year"].max()
    nn["jun_date"] = nn.index.map(lambda d: d.replace(year=tgtyear if d.month >= 6 else tgtyear + 1))
    nn["jun_year_l"] = nn["jun_year"].astype(str) + "/" + ((nn["jun_year"] + 1) % 100).astype(str)
    hue_order = nn["jun_year_l"].unique()
    pal = sns.color_palette(n_colors=len(hue_order))
    maxjundate = nn.loc[nn["jun_year"] == nn["jun_year"].max(), "jun_date"].max()
    if radius:
        nn = nn[abs(nn["jun_date"] - maxjundate) <= radius]
    sns.lineplot(
        nn[nn.index <= ww_completeuntil],
        x="jun_date",
        y="Abwassersignal",
        hue="jun_year_l",
        palette=pal,
        hue_order=hue_order,
        marker=".",
        mew=0,
    )
    sns.lineplot(
        nn[nn.index >= ww_completeuntil],
        x="jun_date",
        y="Abwassersignal",
        hue="jun_year_l",
        palette=pal,
        hue_order=hue_order,
        marker=".",
        mew=0,
        lw=0.5,
        alpha=0.7,
        legend=False,
        markersize=3,
    )

    # ax.set_xlim(left=pd.to_datetime("2022-08-15"), right=pd.to_datetime("2022-12-01"))
    # ax.set_ylim(bottom=0, top=400)

    cov.set_date_opts(ax, showyear=False, week_always=True)
    ax.set_xlim(*ax.get_xlim())
    newyear = date(tgtyear + 1, 1, 1)
    ax.axvline(newyear, color="k", ls="--", lw=1)
    # nn["jun_date"] = nn.index.to_period("A-JUN")
    # display(nn.tail(180))
    ax.set_ylabel("Genkopien pro Tag / EW * 10⁶")
    ax.legend(title="Jahr bzw. Saison")
    ax.set_xlabel(None)
    fig.suptitle("Österreich: Abwasser-Viruslast pro Einwohner je Jahr/Saison, " + NATMON_STAND)
    oldborder = natmon_norm.dropna().index[0].strftime("%d.%m.%y")
    ax.set_title(
        f"Ab {oldborder} ungeglättet (aber Abdeckungs- & Bevölkerungs-normiert), davor geglättete Daten "
        + " | Daten bis: "
        + nn.index[-1].strftime("%a %d.%m.%y")
        + " / "
        + ww_completeuntil.strftime("%d.%m")
        + " vollst."
        + "\nWochengrenzen beziehen sich auf aktuelle Saison ("
        + nn["jun_year_l"].iloc[-1]
        + ")"
    )

    cov.labelend2(
        ax,
        nn,
        "Abwassersignal",
        cats="jun_year",
        x="jun_date",
        shorten=lambda y: str(y % 100),
        colorize=pal,
        ends=(True, False),
    )
    lastlinedata = nn[nn["jun_year_l"] == nn["jun_year_l"].iloc[-1]]
    over_jan = lastlinedata.index[0].year != lastlinedata.index[-1].year
    cov.labelend2(
        ax,
        nn,
        "Abwassersignal",
        cats="jun_year",
        x="jun_date",
        shorten=lambda y: str(y % 100 + 1 if over_jan or y != nn["jun_year"].iat[-1] else y % 100),
        colorize=pal,
        ends=(False, True),
    )
    ax.axhline(
        max(nn["Abwassersignal"].iat[-1], nn.loc[nn.index <= ww_completeuntil, "Abwassersignal"].iat[-1]),
        color=pal[-1],
        lw=0.7,
        zorder=1,
    )
    ax.set_ylim(0)
    cov.stampit(fig, DS_WW)


plt_ww_yoy()

In [ ]:
def plt_ww_csum_yoy(radius=None):
    fig, ax = plt.subplots(layout="constrained")
    blvat = blverlauf_last.query("Bundesland == 'Österreich'")
    nn_old = blvat[blvat["Datum"] < natmon_norm.dropna().index[0]].set_index("Datum")["y"]

    nn = natmon_norm_sm.combine_first(nn_old).to_frame()
    nn["jun_year"] = nn.index.to_period("A-MAY").year - 1
    # nn = nn.query("jun_year > 2021").copy()
    tgtyear = nn["jun_year"].max()
    # nn["jun_date"] = nn.index.map(lambda d: d.replace(year=tgtyear))
    nn["jun_date"] = nn.index.map(lambda d: d.replace(year=tgtyear if d.month >= 6 else tgtyear + 1))
    nn["jun_year_l"] = nn["jun_year"].astype(str) + "/" + ((nn["jun_year"] + 1) % 100).astype(str)
    nn["csum"] = nn.groupby("jun_year")["Abwassersignal"].cumsum()
    hue_order = nn["jun_year_l"].unique()
    pal = sns.color_palette(n_colors=len(hue_order))
    maxjundate = nn.loc[nn["jun_year"] == nn["jun_year"].max(), "jun_date"].max()
    if radius:
        nn = nn[abs(nn["jun_date"] - maxjundate) <= radius]
    sns.lineplot(
        nn,
        x="jun_date",
        y="csum",
        hue="jun_year_l",
        palette=pal,
        hue_order=hue_order,
        mew=0,
    )

    # ax.set_xlim(left=pd.to_datetime("2022-08-15"), right=pd.to_datetime("2022-12-01"))
    # ax.set_ylim(bottom=0, top=400)

    cov.set_date_opts(ax, showyear=False, week_always=True)
    ax.set_xlim(*ax.get_xlim())
    newyear = date(tgtyear + 1, 1, 1)
    ax.axvline(newyear, color="k", ls="--", lw=1)
    # nn["jun_date"] = nn.index.to_period("A-JUN")
    # display(nn.tail(180))
    ax.set_ylabel("Summe Genkopien / EW * 10⁶")
    ax.legend(title="Jahr bzw. Saison")
    ax.set_xlabel(None)
    fig.suptitle("Österreich: Abwasser-Viruslast pro Einwohner je Jahr/Saison, " + NATMON_STAND)
    oldborder = natmon_norm.dropna().index[0].strftime("%d.%m.%y")
    ax.set_title(
        "Daten bis: "
        + nn.index[-1].strftime("%a %d.%m.%y")
        + " / "
        + ww_completeuntil.strftime("%d.%m")
        + " vollst."
        + " | Wochengrenzen beziehen sich auf aktuelle Saison ("
        + nn["jun_year_l"].iloc[-1]
        + ")"
        + " | Selbst geglättete Virenfracht"
    )

    lastlinedata = nn[nn["jun_year_l"] == nn["jun_year_l"].iloc[-1]]
    over_jan = lastlinedata.index[0].year != lastlinedata.index[-1].year
    cov.labelend2(
        ax,
        nn,
        "csum",
        cats="jun_year",
        x="jun_date",
        shorten=lambda y: str(y % 100 + 1 if over_jan or y != nn["jun_year"].iat[-1] else y % 100),
        colorize=pal,
        ends=(False, True),
    )
    ax.yaxis.set_major_formatter(matplotlib.ticker.FuncFormatter(lambda v, _: f"{v:,.0f}".replace(",", " ")))
    ax.axhline(
        max(nn["csum"].iat[-1], nn.loc[nn.index <= ww_completeuntil, "csum"].iat[-1]),
        color=pal[-1],
        lw=0.7,
        zorder=1,
    )
    ax.set_ylim(0)
    cov.stampit(fig, DS_WW)


plt_ww_csum_yoy()

In [ ]:
r_ages = pd.read_csv(
    util.DATAROOT / "covid/ages-ems-extra/R_eff_latest.csv",
    sep=";",
    decimal=",",
    parse_dates=["Datum"],
    index_col="Datum",
)["R_eff"]

In [ ]:
def plt_comp_r_yoy():
    rww_latest = rww.xs(rww.index.unique("FileDate").max(), level="FileDate")["rww"]
    rww_latest = rww_latest[rww_latest.index >= pd.to_datetime("2022-02-01")]
    pdata = rww_latest.combine_first(r_ages).to_frame()
    # display(rww_latest.index[0])
    pdata.loc[rww_latest.index[0] - timedelta(1) : rww_latest.index[0] + timedelta(1)] = None
    # display(pdata[pdata.index >= "2022-01-01"].head(40))
    pdata = pdata[pdata.index > pd.to_datetime("2020-03-25")]
    pdata = pdata[pdata.index <= ww_completeuntil]
    pdx = (pdata.index - pdata.index[0]).days
    # perlen = 700
    # pdata["xperx"] = pdx % perlen
    # pdata["xperi"] = pdx // perlen
    pdata["jun_year"] = pdata.index.to_period("A-MAY").year - 1
    pdata["jun_date"] = pdata.index.map(lambda d: d.replace(year=2022 if d.month >= 6 else 2023))
    pdata["jun_year_l"] = pdata["jun_year"].astype(str) + "/" + ((pdata["jun_year"] + 1) % 100).astype(str)
    hue_order = pdata["jun_year_l"].unique()
    # print(len(hue_order))
    pal = sns.color_palette(n_colors=len(hue_order))

    fig, ax = plt.subplots(layout="constrained")
    cov.stampit(fig, DS_WW + ", " + cov.DS_AGES + ", TU Wien")

    for (lbl, data), hue in zip(pdata.groupby("jun_year_l"), pal, strict=True):
        ax.plot(data["jun_date"], data["rww"], color=hue, label=lbl)
    # display(pdata[pdata.index >= "2022-01-01"].head(40))

    cov.labelend2(
        ax,
        pdata,
        "rww",
        cats="jun_year",
        x="jun_date",
        shorten=lambda y: str(y % 100 if y != 2019 else 20),
        colorize=pal,
        ends=(True, False),
    )
    lastlinedata = pdata[pdata["jun_year_l"] == pdata["jun_year_l"].iloc[-1]]
    over_jan = lastlinedata.index[0].year != lastlinedata.index[-1].year
    cov.labelend2(
        ax,
        pdata,
        "rww",
        cats="jun_year",
        x="jun_date",
        shorten=lambda y: str(y % 100 + 1 if over_jan or y != pdata["jun_year"].iat[-1] else y % 100),
        colorize=pal,
        ends=(False, True),
    )

    ax.set_ylabel("R eff")
    ax.legend(title="Jahr bzw. Saison")
    ax.set_xlabel(None)
    fig.suptitle("Österreich: R eff je Jahr/Saison, " + NATMON_STAND)
    oldborder = rww_latest.dropna().index[0].strftime("%d.%m.%y")
    ax.set_title(
        f"Ab {oldborder} aus Abwasser, davor aus Inzidenz EMS-gemeldeter Fälle"
        + " | Daten bis: "
        + ww_completeuntil.strftime("%a %d.%m.%y")
    )
    ax.axhline(1, color="k", zorder=1)

    cov.set_date_opts(ax, showyear=False)
    ax.axvline(date(2023, 1, 1), color="k", ls="--", lw=1)


plt_comp_r_yoy()

In [ ]:
from scipy.interpolate import splev, splrep
from scipy.signal import savgol_filter


def plt_chng():
    fig, ax = plt.subplots(layout="constrained")
    # cov.set_logscale(ax)
    pnatmon = natmon_norm.dropna()
    ax.plot(pnatmon, marker=".", label="Lineare Interpolation")
    # ax.plot(natmon_norm.dropna().resample("1D").interpolate())
    nn = natmon_norm.dropna().iloc[7:]
    nn_compl = np.log(nn.loc[nn.index <= ww_completeuntil])
    spl = splrep(nn_compl.index, nn_compl.to_numpy(), s=len(nn_compl) * 0.011)  # , w=nnw.iloc[:len(nn_compl)])
    xs = pd.date_range(nn_compl.index[0], nn_compl.index[-1])
    sm = pd.Series(savgol_filter(nn_compl, 14, 3), index=nn_compl.index)
    sm = pd.Series(np.exp(splev(xs, spl)), index=xs)[7:]
    ax.plot(
        sm.loc[sm.index >= pnatmon.index[0]],
        # label="savgol_filter"
        label="spline fit on log",
    )
    r7 = natmon_norm_sm  # nn.ewm(times=nn.index, halflife=timedelta(5)).mean().shift(-3)
    ax.plot(natmon_norm_sm, label="ewm-based")
    cov.set_date_opts(ax, week_always=True)
    with warnings.catch_warnings(action="ignore", category=UserWarning):
        fig.autofmt_xdate()
    fig.suptitle("Österreich: Wöchentliche Abwasserlast, versch. Interpolationen/Curve-Fits [experimentell]")
    ax.legend()

    fig, ax = plt.subplots(layout="constrained")
    ax.plot(natmon_norm_interp.iloc[14:] / natmon_norm_interp.shift(7), label="Lineare Interpolation")
    ax.plot(sm / sm.shift(7), label="savgol_filter")
    ax.plot(r7 / r7.shift(7), label="ewm(halflife=4 d)")
    ax.legend()
    # ax.set_ylim(0)
    ax.axhline(1, color="k")
    cov.set_date_opts(ax)
    with warnings.catch_warnings(action="ignore", category=UserWarning):
        fig.autofmt_xdate()
    fig.suptitle("Österreich: Wöchentliche Abwasserlast-Änderung, versch. Interpolationen/Curve-Fits [experimentell]")
    ax.yaxis.set_major_locator(matplotlib.ticker.MultipleLocator(0.1))


if plt_experimental:
    plt_chng()

In [ ]:
from itertools import pairwise


def plt_ww_all_different():
    blvat = blverlauf.query("Bundesland == 'Österreich'").copy()
    assert len(blvat) > 0
    # display(blvat)
    grp = blvat.groupby("FileDate")
    samevals = [False] + [
        len(fd1) == len(fd2) and (fd1.to_numpy() == fd2.to_numpy()).all() for (_, fd1), (_, fd2) in pairwise(grp["y"])
    ]
    samevals = pd.Series(samevals, index=grp.indices, dtype=bool, name="same")
    blvat = blvat.merge(samevals, left_on="FileDate", right_index=True)

    blvat["FileDate"] = blvat["FileDate"].dt.tz_localize(None)  # It seems that sns/hue has a problem with TZ
    ax = sns.lineplot(
        blvat[~blvat["same"]],
        palette="cet_glasbey_dark",
        x="Datum",
        y="y",
        mew=0,
        hue="FileDate",
        legend=False,
        hue_norm=matplotlib.colors.Normalize(),
    )
    fig = ax.figure
    fig.suptitle("Abwasser: Alle Datenstände Österreich", y=0.93)


if False:
    plt_ww_all_different()

In [ ]:
# display(blv1)
def draw_blv1(blv1=blv1):
    blv1 = blv1[blv1.index.get_level_values("Datum") >= blv1.index.get_level_values("Datum").max() - timedelta(120)]
    fig, ax = plt.subplots()
    cov.set_logscale(ax)
    sns.lineplot(blv1["y"].reset_index(), x="Datum", y="y", hue="Bundesland", mew=0, marker=".")

    ax.get_legend().remove()
    cov.set_date_opts(ax, blv1.index.get_level_values("Datum"), week_always=True)
    ax.set_ylabel("Genkopien pro Tag / EW * 10⁶")

    fig.suptitle("Österreich : SARS-CoV-2 im Abwasser, nur Erstmeldungen, logarithmische Skala", y=1.04)
    fig.legend(
        *cov.sortedlabels(ax, blv1.reset_index(), "y", fmtval=lambda v: f"{v:.0f}"),
        ncol=5,
        frameon=False,
        loc="upper center",
        bbox_to_anchor=(0.5, 0.965),
    )
    ax.set_title(
        abw_tinfo(blv1) + ", Daten bis " + blv1.index.get_level_values("Datum").max().strftime("%a %d.%m.%y"), y=1.09
    )
    cov.labelend2(ax, blv1.reset_index(), ycol="y", colorize=sns.color_palette(), ends=(True, True))
    # print(blv1["FileDate"].max())


draw_blv1()

In [ ]:
def plt_ww_longsum():
    rwidth = 32

    ww_sum = natmon_norm.reindex(
        pd.date_range(natmon_norm.index[0], natmon_norm.index[-1] + timedelta(30))
    ).interpolate().ewm(halflife=rwidth, min_periods=rwidth * 1.25).sum().dropna() * (10 / 100_000 / 7)

    rww_latest = rww.xs(rww.index.unique("FileDate").max(), level="FileDate")["rww"]
    rww_latest = rww_latest[rww_latest.index >= ww_sum.index[0]]
    rww_latest = rww_latest[rww_latest.index < ww_completeuntil]

    fig, ax = plt.subplots(layout="constrained")
    fig.suptitle(
        f"Österreich: ewm(halflife={rwidth} Tage)-Summe Infektionen/EW, Grobschätzung aus Abwasserlast [experimentell]"
    )
    ax.plot(ww_sum[: natmon_norm.index[-1]], color="C0")
    ax.plot(ww_sum[natmon_norm.index[-1] :], color="C0", lw=1, ls="--")
    cov.set_date_opts(ax, showyear=True)
    cov.set_percent_opts(ax)
    cov.stampit(fig, DS_WW)
    ax2 = ax.twinx()
    ax2.grid(False)
    ax2.axhline(1, color="C1", lw=1)
    ax2.plot(rww_latest, color="C1")
    with warnings.catch_warnings(action="ignore", category=UserWarning):
        fig.autofmt_xdate()


if plt_experimental or True:
    plt_ww_longsum()

## Varianten

Varianten kommen aus zwei unabhängigen Quellen:

1. Abwasser-Sequenzierung:
    * Daten von: CeMM (Forschungszentrum für Molekulare Medizin) der ÖAW (Österreichischen Akademie der Wissenschaften); in Zusammenarbeit mit o.g. Abwassermonitoring-Akteuren
    * Daten-URL: <https://abwassermonitoring.at/dashboard/> (
    * Daten-Aktualisierung: Unregelmäßig; zuletzt wöchentlich, Montag vormittags
2. AGES / Klinische Sequenzen:
    * Daten von: AGES
    * Daten-URL: <https://www.ages.at/mensch/krankheit/krankheitserreger-von-a-bis-z/coronavirus#c26136> (siehe auch "Downloads")
    * Daten-Aktualisierung: wöchentlich, Dienstag nachmittags

In [ ]:
with (util.DATAROOT / "covid/alias_key.json").open() as f:
    alias_key_raw = json.load(f)
alias_key = {k: v if isinstance(v, str) else k for k, v in alias_key_raw.items()}
fullname_key = {v: k for k, v in alias_key.items()}


def expand_variant(k):
    if "." not in k:
        return alias_key.get(k, k)
    pdot = k.index(".")
    return alias_key[k[:pdot]] + k[pdot:]


def short_variantname(k):
    parts = k.split(".")
    for i in range(len(parts) - 1, 0, -1):
        pfx = ".".join(parts[:i])
        pfx_alias = fullname_key.get(pfx)
        # print(k, i, pfx, pfx_alias)
        if pfx_alias:
            return pfx_alias + "." + ".".join(parts[i:])
    return k

In [ ]:
# import http.client
import email.utils
from datetime import timezone
import traceback

cov = reload(cov)


def plt_variants(seqdistr, seqdate, stamp, *, title_infix, minimal=False):
    # display(nullvars)
    nseqvars = seqdistr["variant"].nunique()
    # seqdistr["share"] /= 100

    palette = sns.color_palette(
        "cet_glasbey_dark" if nseqvars > 20 else "tab20" if nseqvars > 10 else "tab10", n_colors=nseqvars
    )

    blv_at_last_y = natmon_norm_sm
    dt_end = seqdistr["Datum"]  # + timedelta(6)
    seqdistr = seqdistr.merge(blv_at_last_y.rolling(7).mean().rename("load"), left_on=dt_end, right_index=True, how="left")
    seqdistr["load"] *= seqdistr["share"]
    # display(seqdistr)

    dt_end = seqdistr["Datum"]  # + timedelta(6)  # Need to recalc b/c of index change
    # seqdistr["Datum"] += timedelta(3)  # Week center (last date is start of last day)
    # display(seqdistr)

    ncol = 7 if seqdistr["variant"].nunique() > 10 else 5

    fig = plt.figure()
    ax = sns.lineplot(seqdistr, x="Datum", y="share", hue="variant", marker=".", mew=0, palette=palette)
    cov.set_date_opts(ax)
    cov.set_logscale(ax)
    cov.set_percent_opts(ax, decimals=1)
    ax.get_legend().remove()
    cov.labelend2(
        ax,
        seqdistr,
        "share",
        cats="variant",
        shorten=lambda v: v,
        colorize=palette,
        ends=(True, True),
    )
    fig.legend(loc="upper center", ncol=ncol, frameon=False, bbox_to_anchor=(0.5, 0.97))
    fig.suptitle(f"Österreich: SARS-CoV-2-{title_infix}Variantenverteilung, logarithmisch, Stand {seqdate}", y=1)
    ax.set_ylabel("Anteil")
    ax.set_xlabel("Probedatum")
    cov.stampit(fig, stamp)

    if minimal:
        return

    fig, ax = plt.subplots()
    ax = sns.lineplot(seqdistr, x="Datum", y="load", hue="variant", marker=".", mew=0, ax=ax, palette=palette)
    cov.set_logscale(ax)
    ax.yaxis.set_major_formatter(matplotlib.ticker.FormatStrFormatter("%.1f"))
    ax.get_legend().remove()
    ax.plot(
        blv_at_last_y[blv_at_last_y.index.get_level_values("Datum") >= seqdistr["Datum"].min()],
        color="k",
        label="Summe",
    )
    cov.labelend2(
        ax,
        seqdistr,
        "load",
        cats="variant",
        shorten=lambda v: v,
        colorize=palette,
        ends=(True, True),
    )
    fig.legend(loc="upper center", ncol=ncol, frameon=False, bbox_to_anchor=(0.5, 0.97))
    fig.suptitle("Österreich: SARS-CoV-2-Varianten-Abwasserlast, logarithmisch, Stand Varianten: " + seqdate, y=1)
    ax.set_ylabel("Genkopien pro Tag / EW * 10⁶")

    cov.set_date_opts(ax)
    ax.set_xlabel("Probedatum")
    ax.axvline(date.today(), color="k", ls="--")
    cov.stampit(fig, stamp)

    ###########

    # display([pdata[c] for c in pdata.columns])
    palette = sns.color_palette(
        "cet_glasbey" if nseqvars > 20 else "tab20" if nseqvars > 10 else "tab10", n_colors=nseqvars
    )
    fig, ax = plt.subplots()
    ax.stackplot(
        seqdistr["Datum"].unique(),
        *[seqdistr.loc[seqdistr["variant"] == c, "load"].to_numpy() for c in seqdistr["variant"].unique()],
        labels=seqdistr["variant"],
        lw=0,
        colors=palette,
    )
    seqdistr["cs"] = seqdistr.groupby("Datum")["load"].cumsum()
    seqdistr["cc"] = seqdistr.groupby("Datum")["cs"].transform(
        lambda cs: pd.concat([pd.Series([0]), cs]).rolling(2).mean().iloc[1:]
    )
    seqdistr["cc"] = seqdistr["cc"].where(seqdistr["load"] > 0)
    # display(seqdistr)
    cov.set_date_opts(ax)
    # cov.set_logscale(ax)
    ax.yaxis.set_major_formatter(matplotlib.ticker.FormatStrFormatter("%.1f"))
    # ax.get_legend().remove()
    evars = cov.filterlatest(seqdistr).loc[lambda sd: sd["load"] > 0]["variant"]
    cov.labelend2(
        ax, seqdistr, "cc", cats="variant", shorten=lambda v: v, colorize=palette, ends=(True, True), border_only=True
    )
    ax.set_axisbelow(False)
    fig = ax.figure
    fig.legend(loc="upper center", ncol=ncol, frameon=False, bbox_to_anchor=(0.5, 0.97))
    fig.suptitle("Österreich: SARS-CoV-2-Varianten-Abwasserlast, Stand Varianten: " + seqdate, y=1)
    ax.set_ylabel("Genkopien pro Tag / EW * 10⁶")
    ax.set_xlabel("Probedatum")
    cov.stampit(fig, stamp)


FLIP_VARS = ["GK", "HK.3", "DV.7.1", "JD.1.1", "JG.3"]
WATCHED_SELECTORS = {
    "BA.2*": ["BA.2"],
    "BA.5*+BA.4*": ["BA.4", "BA.5"],
    "XBB*": ["XBB"],
    "XBB.1.5*": ["XBB.1.5"],
    "EG.5* „Eris“": ["EG.5"],
    "BA.2.86* „Pirola“": ["BA.2.86"],
    "Häufige FLips": FLIP_VARS,
}


def summarize_variants_prepare(counts, selectors=WATCHED_SELECTORS, otherlabel=None):
    counts = counts.reset_index()
    counts["var_fullname"] = counts["variant"].map(expand_variant) + "."
    counts["var_key"] = counts["variant"] if otherlabel is None else otherlabel

    def match_var(vname):
        return counts["var_fullname"].str.startswith(expand_variant(vname) + ".")

    for selector_key, selector_parents in selectors.items():
        mask = None
        for selector_parent in selector_parents:
            mask = match_var(selector_parent) if mask is None else mask | match_var(selector_parent)
        counts.loc[mask, "var_key"] = selector_key
    counts["var_fullname"] = counts["var_fullname"].str.removesuffix(".")
    return counts


def summarize_variants_finalize(prepared_counts):
    return (
        prepared_counts.drop(columns="variant", errors="raise")
        .rename(columns={"var_key": "variant"}, errors="raise")
        .groupby(["variant", "Datum"], sort=False)["share"]
        .sum()
    )


def summarize_variants_and_print(counts):
    counts = summarize_variants_prepare(counts)
    display(
        counts.loc[counts["Datum"] == counts["Datum"].max()]
        .drop(columns="index", errors="ignore")
        .sort_values("share", ascending=False)
        .query("share > 0")
        .drop(columns=["week", "Datum"], errors="ignore")
        .set_index("variant")
        .style.format(subset=["share"], formatter=lambda v: f"{v:.2%}")
        .set_properties(**{'text-align': 'left'})
    )
    return summarize_variants_finalize(counts)


def plt_variants_ages():
    fname = tuple(sorted((util.DATAROOT / "covid/ages-varianten").glob("Varianten_????-??-??.csv")))[-1]
    seqdate = datetime.strptime(fname.stem, "Varianten_%Y-%m-%d").replace(tzinfo=util.TZ_AT).strftime("%d.%m.%y")
    counts = (
        pd.read_csv(fname, sep=";")
        .rename(columns={"Lineage": "variant"}, errors="raise")
        .melt("variant", var_name="week", value_name="n")
    )
    counts["Datum"] = pd.to_datetime(counts["week"] + "1", format="%G-W%V%u")
    print("Letztes Sequenzierungsdatum (AGES-Proben; KW ab):", counts["Datum"].iloc[-1])
    counts["Datum"] += timedelta(2)  # Move to Wednesday
    counts["variant"].replace("Proben insgesamt", "Proben gesamt", inplace=True)
    counts = counts[counts["Datum"] >= counts["Datum"].max() - timedelta(90)]
    counts = counts.set_index(["variant", "Datum"]).sort_index()

    n_total = counts.xs("Proben gesamt")["n"]
    n_failed = counts.xs("nicht auswertbar")["n"]
    counts = counts.loc[counts.index.get_level_values("Datum") >= (n_total[n_total > 0].index.min())]

    fig, ax = plt.subplots(layout="constrained")
    ax.plot(n_total, marker="o", color="C0", label="Insgesamt zur Sequenzierung gebrachte Proben")
    ax.plot(n_total - n_failed, marker="s", color="C1", label="Davon auswertbar")
    ax.legend(loc="lower center", bbox_to_anchor=(0.5, 1), frameon=False, ncol=2)
    fig.suptitle(f"AGES-sequenzierte SARS-CoV-2-Proben, Stand {seqdate}")
    lastcompldate = matplotlib.dates.date2num(counts.index.unique("Datum")[-3]) + 1
    ax.set_xlim(*ax.get_xlim())
    ax.axvspan(lastcompldate, ax.get_xlim()[1], color="k", alpha=0.2)
    ax.set_ylim(bottom=0)
    cov.stampit(fig, cov.DS_AGES)
    cov.set_date_opts(ax)

    counts.drop(index=["Proben gesamt", "nicht auswertbar"], errors="raise", inplace=True)
    counts["share"] = counts["n"] / (n_total - n_failed)
    counts = summarize_variants_and_print(counts)
    plt_variants(counts.reset_index(), seqdate, cov.DS_AGES, minimal=True, title_infix="AGES-")
    ax = plt.gca()
    ax.set_xlim(*ax.get_xlim())
    ax.axvspan(lastcompldate, ax.get_xlim()[1], color="k", alpha=0.2)
    # display(counts)


def summarize_variants_dyn(seqdistr):
    seqdistr = seqdistr.reset_index()
    recent = seqdistr[seqdistr["Datum"] > seqdistr["Datum"].max() - timedelta(20)]

    def isrising(s):
        return (
            s.iat[0] * 1.1 < s.iloc[1:].max()
            or s.iat[1] > 0
            and (s.iat[1] / s.iat[0] >= 1.2 or s.iat[2] / s.iat[1] >= 1.2)
        )

    raiserecent = recent.groupby("variant")["share"].agg(isrising)[lambda s: s].index
    largeshare = seqdistr.groupby("variant")["share"].agg(lambda s: (s > 0.1).any())[lambda s: s].index
    watchlist = [v for v in seqdistr["variant"].unique() if v.startswith("JN") or (v + ".").startswith("BA.2.86.")]
    included = raiserecent.union(largeshare).union(watchlist)
    included_mask = seqdistr["variant"].isin(included)

    if True:
        inclfullnames = {v: expand_variant(v) + "." for v in seqdistr.loc[included_mask, "variant"].unique()}
        groupkeys = {
            v: next((p for p, pf in inclfullnames.items() if expand_variant(v).startswith(pf)), "Sonstige")
            for v in seqdistr.loc[~included_mask, "variant"].unique()
        }
        restfulln = os.path.commonprefix([expand_variant(v) + "." for v, k in groupkeys.items() if k == "Sonstige"])
        # display(f"{groupkeys=}, {restfulln=}")
        restfulln = "Sonstige" if "." not in restfulln else short_variantname(restfulln[: restfulln.rindex(".")])
        groupkeys = {v: k if k != "Sonstige" else restfulln for v, k in groupkeys.items()}

        seqdistr["var_fullname"] = seqdistr["variant"].map(expand_variant)
        seqdistr.loc[~included_mask, "variant"] = seqdistr.loc[~included_mask, "variant"].map(groupkeys)
        seqdistr.loc[
            (seqdistr["variant"] != "Sonstige") & seqdistr["variant"].isin(groupkeys.values()), "variant"
        ] += "+"
        seqdistr.sort_values(["var_fullname", "variant"], kind="stable", inplace=True)
    else:
        seqdistr.loc[~included_mask, "variant"] = "Sonstige"

    # display(seqdistr[seqdistr["Datum"] == seqdistr["Datum"].max()])
    return seqdistr.groupby(["Datum", "variant"], sort=False).sum().reset_index()


def plt_variants_ww():
    seqdate = (
        util.fdate_from_fname(Path(util.DATAROOT / "covid/abwassermonitoring/dl_natmon_variants_latest.csv").readlink())
        .astimezone(util.TZ_AT)
        .strftime("%d.%m.%y")
    )

    seqdistr = pd.read_csv(
        util.DATAROOT / "covid/abwassermonitoring/dl_natmon_variants_latest.csv", sep=";", parse_dates=["Datum"]
    )
    seqdistr.rename(
        inplace=True,
        errors="raise",
        columns={
            "Variante": "variant",
            "Anteil [%]": "share",
        },
    )
    print("Letztes Sequenzierungsdatum (Abwasser):", seqdistr["Datum"].iloc[-1])
    rem100 = (1 - seqdistr.groupby("Datum")["share"].sum()).reset_index()
    rem100["variant"] = "Rest auf 100%"
    # display(rem100)
    seqdistr = pd.concat([seqdistr, rem100]).reset_index(drop=True)
    seqdistr_all = seqdistr
    if True:
        seqdistr = seqdistr[seqdistr["Datum"] >= seqdistr["Datum"].max() - timedelta(90)]
        notnullvars = seqdistr.groupby("variant")["share"].agg(lambda s: (s > 0).any())[lambda s: s].index
        seqdistr = seqdistr[seqdistr["variant"].isin(notnullvars)]

        plt_variants(summarize_variants_dyn(seqdistr).reset_index(), seqdate, DS_WW_VAR, title_infix="Abwasser-")
        plt_variants(
            summarize_variants_and_print(seqdistr).reset_index(),
            seqdate,
            DS_WW_VAR,
            title_infix="Abwasser-",
            minimal=True,
        )
        plt.gca().set_title("ACHTUNG: Häufige FLips aufgrund grober Variantenaufteilung untererfasst", y=0.95)

    newdistr_w = summarize_variants_finalize(
        summarize_variants_prepare(
            seqdistr_all,
            {
                "BA.2.86* „Pirola“ (inkl. JN.1)": ["BA.2.86"],
                # "JN.1": ["JN.1"],
                "XBB*": ["XBB"],
                # "EG.5* ohne FLip": ["EG.5"],
                # "FLip": FLIP_VARS
            },
            otherlabel="Rest",
        )
    )
    mdate = newdistr_w.index.unique("Datum").max() - timedelta(400)

    def upsample_vars(s):
        v = s.index.get_level_values("variant")[0]
        s = s.droplevel(0).resample("1D").interpolate()
        # s.loc[:(newdistr_w.xs(v).loc[newdistr_w > 0]).index[0] - timedelta(7)] = 0
        return s.loc[s.index >= mdate]

    newdistr = newdistr_w.groupby("variant").apply(upsample_vars)
    # display(newdistr)
    fig, ax = plt.subplots()
    # cov.set_logscale(ax)
    ax.yaxis.set_major_formatter(matplotlib.ticker.StrMethodFormatter("{x:.1f}"))
    for vn, ys in newdistr.groupby("variant", sort=False):
        # display(vn, ys.droplevel(0))
        pys = ys.droplevel(0)
        pys = pys.loc[pys[pys > 0].index[0] - timedelta(1) :]
        ax.plot(pys * natmon_norm_sm, label=vn)
    ax.set_ylabel("Abwasser-Virus-Genkopien pro Tag / EW * 10⁶")
    # ax.set_ylim(bottom=0.1)
    ax.legend()
    cov.set_date_opts(ax)
    fig.suptitle("Österreich: SARS-CoV-2-Abwasserlast je Variante", y=0.96)
    ax.set_title(ABW_TINFO + " / Sequenzen: " + seqdate)
    ax.set_ylim(bottom=0)
    cov.stampit(fig, DS_WW_VAR)


plt_variants_ww()
plt_variants_ages()

# Medikamentenmangel

* Daten von: Österreichiches Bundesamt für Sicherheit im Gesundheitswesen (BASG)
* Daten-URLs:
  * <https://medicineshortage.basg.gv.at/vertriebseinschraenkungen> (Webansicht und xlsx)
  * <https://www.basg.gv.at/fuer-unternehmen/datenbereitstellung-vertriebseinschraenkungen> (xml)
* Daten-Aktualisierung: Täglich (mit sporadischen Aussetzern, v.a. an Nicht-Werktagen)
* [Mehr Daten-Infos](https://github.com/zeitferne/covidat-data/blob/main/docs/basg-medicineshortage.md)

Der Status *"teilweise verfügbar"* wird vom BASG nicht verwendet, sondern wird von dieser Analyse vergeben sobald mindestens eine Packungsgröße des Mangel-Medikamentes (der selben Dosisstärke, etc.) uneingeschränkt verfügbar ist. Beim BASG wird dies als "eingeschränkt verfügbar" geführt. Der Status "eingeschränkt verfügbar" wird hier nur verwendet wenn mindestens eine Packungsgröße tatsächlich "eingeschränkt verfügbar" gemeldet wurde und das Medikament nicht "teilweise verfügbar" ist.

In [ ]:
def labelavail(avail):
    return avail.replace("verfügbar gemäß §4 (1)", "§4(1) (~Mangel entgegen Herstellerangabe)").replace(
        "teilweise verfügbar", "gewisse Größen voll verfügbar (Verwechslung mit eing. verf. möglich)"
    )

In [ ]:
med_x = pd.read_csv(util.COLLECTROOT / "medshort/medshort_ex.csv", encoding="utf-8", sep=";").query(
    "Datum >= '2022-12-01' and (SourceFormat == 'xml' or Datum <= '2023-06-25')"
)
# display(med_x.query("Availability == 'verfügbar'"))
med_x = med_x.query("Availability != 'verfügbar'")
med_x["Datum"] = pd.to_datetime(med_x["Datum"], format="%Y-%m-%d")
med_x["Datum"].freq = "D"
# med_x["Datum_n"] = matplotlib.dates.date2num(med_x["Datum"])
AvailC = pd.CategoricalDtype(
    reversed(["nicht verfügbar", "eingeschränkt verfügbar", "verfügbar gemäß §4 (1)", "teilweise verfügbar"]),
    ordered=True,
)
med_x["Availability"] = med_x["Availability"].astype(AvailC)
med_x.sort_values(["Availability"], kind="stable", inplace=True, ascending=False)
med_x.sort_values(["Datum"], kind="stable", inplace=True, ascending=True)


def med_stamp(dt):
    return dt.strftime("Stand %d.%m.%y")


MED_STAMP = med_stamp(med_x.iloc[-1]["Datum"])


def med_longstamp(dt):
    return "Darstellung: @zeitferne | Daten: medicineshortage.basg.gv.at/vertriebseinschraenkungen, " + med_stamp(dt)


MED_LONGSTAMP = med_longstamp(med_x.iloc[-1]["Datum"])

# display(med_x)
med_xh = med_x.query("Usage == 'Human'").reset_index(drop=True)
# ax = sns.area(med_xh, x="Datum", y="N", hue="Availability", marker="o")
# ax = med_xh.pivot(index="Datum", columns="Availability", values="N").resample("D").plot.bar(stacked=True)
# cov.set_date_opts(ax)
# ax.set_ylim(bottom=0)
fig, ax = plt.subplots()
ax.set_axisbelow(False)
# ax.plot(medshort["NLimitedMeds"], marker="o", color="grey",
#        label="Anzahl Einträge insgesamt, inkl. Veterinär: " + str(medshort["NLimitedMeds"].iloc[-1]), zorder=1)
pltr = cov.StackBarPlotter(
    ax, pd.date_range(start=med_xh["Datum"].min(), end=med_xh["Datum"].max(), freq="1D"), lw=0, width=1
)
ax.set_prop_cycle(color=sns.color_palette("plasma", n_colors=med_x["Availability"].nunique()))
for avail, avail_d in med_xh.groupby("Availability", sort=False, observed=False):
    # display(avail_d)
    pltr(
        avail_d.set_index("Datum")["N"].resample("1D").interpolate(limit=1).to_numpy(),
        label=labelavail(avail) + ": " + str(avail_d["N"].iloc[-1]),
    )


def decorate_medshort(ax, titley, supy, extralabel=""):
    ax.figure.suptitle("Medikamentenmangel in Österreich", y=supy)
    cov.set_date_opts(ax, showyear=True)
    ax.axvline(
        pd.to_datetime("2023-06-23T12:00:00"),
        color="k",
        ls=":",
        label="Umstellung eigener Datensammlung von xlsx auf xml",
    )
    ax.set_title(MED_LONGSTAMP + extralabel, y=titley)
    ax.set_ylabel("Anzahl Human-Medikamente")


# ax.figure.autofmt_xdate()
decorate_medshort(ax, 1.05, 1.01, "")
ax.legend(loc="lower left", title="Anzahl Human-Medikamente je Kategorie")
# med_xh

if False:
    ax.text(
        0.5,
        1.01,
        "Falls ein Medikament in allen beschränkten Dosisstärken in allen Packungsgrößen beschränkt ist, "
        "aber eine andere Dosisstärke uneingeschränkt verfügbar wäre,\n"
        "wird stattdessen wegen Limitierungen in der Daten-Analyse fälschlicherweise "
        "die stärkere Beschränkung angegeben.",
        fontsize="x-small",
        ha="center",
        transform=ax.transAxes,
    )
display(cov.filterlatest(med_xh).set_index(["Datum", "Usage", "Availability"])[["N"]])
print("In Summe:", cov.filterlatest(med_xh)["N"].sum())
# display(med_xh[med_xh["Datum"] == med_xh["Datum"].min()])

In [ ]:
def plt_medshort_sep():
    pdata = med_xh.copy()
    med_xh["Availability"] = labelavail(med_xh["Availability"])
    ax = sns.lineplot(med_xh, x="Datum", y="N", hue="Availability", palette="plasma_r")
    ax.get_legend().remove()
    decorate_medshort(ax, supy=1.065, titley=1.14)
    fig = ax.figure
    fig.legend(
        *cov.sortedlabels(ax, med_xh, "N", "Availability", fmtval=lambda v: f"{v:.0f}"),
        frameon=False,
        ncol=2,
        loc="upper center",
        bbox_to_anchor=(0.5, 1),
    )
    ax.set_ylim(bottom=0)
    ax.set_xlabel(None)


plt_medshort_sep()

In [ ]:
azr = collectshortage.load_azr()


def load_sx(tstamp="latest"):
    sx = collectshortage.load_veasp_xml(
        util.DATAROOT / f"basg-medicineshortage/VertriebseinschraenkungenASP_{tstamp}.xml", azr, only_statagg=False
    )
    sx["Status"] = sx["Status"].astype(AvailC)
    return sx


sx = load_sx()
sx_oldsuffix = sorted(Path(util.DATAROOT / "basg-medicineshortage").glob("VertriebseinschraenkungenASP_*_*.xml"))[
    -2
].stem.split("_", 1)[1]
sx_old = load_sx(sx_oldsuffix)

In [ ]:
MED_STAMP_OLD = med_stamp(
    datetime.strptime(
        sx_oldsuffix,
        util.DL_TSTAMP_FMT,
    )
    .replace(tzinfo=UTC)
    .astimezone(util.TZ_AT)
)

In [ ]:
if False:
    soon_back = sx[
        (sx["Datum_voraussichtliche_Wiederbelieferung"] > datetime.now() - timedelta(14))
        & (sx["Datum_voraussichtliche_Wiederbelieferung"] < datetime.now() + timedelta(14))
    ][
        [
            "Name",
            "Melder",
            "Status",
            "Wirkstoffe",
            "Grund",
            "Datum_voraussichtliche_Wiederbelieferung",
            "Datum_letzte_Aenderung",
        ]
    ].sort_values(
        "Datum_voraussichtliche_Wiederbelieferung"
    )
    print(len(soon_back), "Medikamente sollten bald (+/- 2 Wochen) wieder Lieferungen erhalten")
    display(soon_back)

In [ ]:
def plt_med_wieder_ax(sx, ax, centerdate, stamp):
    tod = sx["Datum_letzte_Aenderung"].max()

    ax.set_title(stamp)
    ax.set_ylabel("Anzahl versch. Medikamente")
    pdata = sx[
        sx["Datum_voraussichtliche_Wiederbelieferung"].between(centerdate - timedelta(60), centerdate + timedelta(200))
    ]
    # sx0 = sx.assign(Datum_voraussichtliche_Wiederbelieferung_n=matplotlib.dates.date2num(sx["Datum_voraussichtliche_Wiederbelieferung"]))
    sns.histplot(
        pdata,
        x="Datum_voraussichtliche_Wiederbelieferung",
        hue="Status",
        multiple="stack",
        binwidth=7,
        palette="plasma_r",
        ax=ax,
    )
    # cov.set_date_opts(ax, showyear=True)
    # ax.tick_params(rotation=90)
    ax.axvline(tod, color="k", ls="--")


def plt_med_wieder():
    fig, axs = plt.subplots(nrows=2, sharex=True, sharey=True)
    fig.suptitle("Voraussichtliche Medikamenten-Wiederbelieferungen je Kalenderwoche", y=0.95)
    centerdate = sx["Datum_letzte_Aenderung"].max()
    plt_med_wieder_ax(sx, axs[0], centerdate, MED_STAMP)
    plt_med_wieder_ax(sx_old, axs[1], centerdate, MED_STAMP_OLD)
    axs[1].set_xlabel("Kalenderwoche der vorraussichtlichen Wiederbelieferung")
    axs[1].get_legend().remove()
    cov.set_date_opts(axs[1])
    cov.stampit(fig, "BASG")


if plt_experimental:
    plt_med_wieder()

## Änderungen bei Medikamentenverfügbarkeit

In [ ]:
def print_med_changes():
    # See git history (9bf30f935c07cc6ea290dfb0776bb3b656737007) for extended table display
    cols = ["Status"]
    sxcmp = sx.set_index("Name")[cols].join(sx_old.set_index("Name")[cols], rsuffix=" vorher", how="outer")
    sxcmp = sxcmp.loc[sxcmp["Status"] != sxcmp["Status vorher"]]
    # sx.set_index("Name")[cols].compare(sx_old.set_index("Name")[cols], keep_equal=True)

    rm_mask = pd.isna(sxcmp["Status"])
    sxrm = sxcmp[rm_mask]
    new_mask = pd.isna(sxcmp["Status vorher"])
    sxnew = sxcmp[new_mask]
    sxchg_mask = ~new_mask & ~rm_mask
    sxchg = sxcmp[sxchg_mask]

    display(
        Markdown(
            f"{len(sxcmp)} insgesamt mit geändertem Status im Vergleich {MED_STAMP_OLD} (vorher) mit {MED_STAMP} (aktuell), davon:\n"
            + f"\n* {len(sxrm)} von Mangelliste entfernt (d.h. wieder uneingeschränkt verfügbar oder ggf. dauerhaft aus Register gestrichen): "
            + ", ".join(
                f"‒{v} {k}" for k, v in sxrm["Status vorher"].value_counts().pipe(lambda s: s[s > 0]).to_dict().items()
            )
            + f"\n* {len(sxnew)} neu auf Mangelliste gekommen: "
            + ", ".join(
                f"+{v} {k}" for k, v in sxnew["Status"].value_counts().pipe(lambda s: s[s > 0]).to_dict().items()
            )
            + f"\n* die restlichen {len(sxchg)} mit geänderter Verfügbarkeit:"
        )
    )
    display(
        sxchg.reset_index()
        .groupby(["Status vorher", "Status"], observed=True)
        .count()
        .query("Name > 0")
        .rename(columns={"Name": "Anzahl"})
    )


print_med_changes()

In [ ]:
def plt_med_reasons_cnt():
    # fig, axs = plt.subplots(ncols=2, sharey=True, gridspec_kw={'width_ratios':[2,1])
    fig, ax = plt.subplots(figsize=(5, 5))
    pdata = sx.query("Status not in ('verfügbar',)").copy()
    pdata["Grund"].fillna("?", inplace=True)
    # ax = sx0.value_counts("Grund", ascending=True, dropna=False).plot.barh()
    top = pdata.value_counts("Grund")
    pdata = pdata.sort_values(by="Grund", key=lambda ks: ks.map(top.index.get_loc))
    ax = sns.histplot(
        pdata,
        y="Grund",
        hue="Status",
        multiple="stack",
        # dodge=False,
        stat="count",
        discrete=True,
        palette="plasma_r",
    )
    # ax.bar_label(ax.containers[0])
    fig = ax.figure
    fig.suptitle("Ans BASG gemeldeter Grund für Mangel (alle Verfügbarkeits-Kategorien)", y=0.96, x=0)
    ax.set_title(MED_LONGSTAMP, x=0)

    # ax = sx.query("Status == 'nicht verfügbar'").value_counts("Grund", ascending=True).plot.barh(ax=axs[1])
    # ax.bar_label(ax.containers[0])
    ax.set_xlabel("Anzahl verschiedener Medikamente")


plt_med_reasons_cnt()

In [ ]:
def plt_med_nv_reasons_count():
    fig, ax = plt.subplots(figsize=(5, 5))
    ax = sx.query("Status == 'nicht verfügbar'").value_counts("Grund", ascending=True, dropna=False).plot.barh()
    ax.bar_label(ax.containers[0])
    fig = ax.figure
    fig.suptitle("Ans BASG gemeldeter Grund für Mangel (nur \"nicht verfügbar\")", y=0.96, x=0)
    ax.set_title(MED_LONGSTAMP, x=0)
    ax.set_xlabel("Anzahl verschiedener Medikamente")


if plt_experimental:
    plt_med_nv_reasons_count()

In [ ]:
def plt_med_nv_em_wirkstoff_count():
    fig, ax = plt.subplots(figsize=(10, 7))
    pdata = sx.explode("Wirkstoffe")
    anz = 5
    topws = pdata.value_counts("Wirkstoffe", dropna=False).where(lambda x: x >= anz).dropna().index
    pdata = pdata.loc[pdata["Wirkstoffe"].isin(topws)]
    pdata = pdata.sort_values(by="Wirkstoffe", key=lambda ks: ks.map(topws.get_loc))
    ax = sns.histplot(
        pdata,
        y="Wirkstoffe",
        discrete=True,
        stat="count",
        hue="Status",
        multiple="stack",
        palette="plasma_r",
    )
    fig = ax.figure
    fig.suptitle(f"Mangel-Medikamente je Wirkstoff (Mehrfachzählung bei mehreren) und Status (nur Anzahl >= {anz})")
    ax.set_xlabel("Anzahl verschiedener Medikamente (Mehrfachzählung bei mehreren)")
    loc = matplotlib.ticker.AutoLocator()
    loc.set_params(integer=True)
    ax.xaxis.set_major_locator(loc)
    sns.move_legend(
        ax,
        "lower center",
        bbox_to_anchor=(0.5, 1),
        ncol=5,
        title=None,
        frameon=False,
    )
    ax.set_title(MED_LONGSTAMP, y=1.05)


plt_med_nv_em_wirkstoff_count()

In [ ]:
GRUND_DEFINITION = (
    (
        "Markt",
        (
            "Vorübergehende Einstellung des Inverkehrbringens",
            'Regulatorische Änderungen',
            'Produktionstransfer zu einem alternativen Hersteller',
            'Wechsel des Zulassungsinhabers',
            'Versorgungspriorisierung anderer Länder',
            'Änderung der Packungsgröße',
            'Konkursverfahren des Herstellers oder des Zulassungsinhabers',
        ),
    ),
    (
        'Verknappung des Wirkstoffes',
        ('Verknappung des Wirkstoffes',),
    ),  # Könnte Kapzität, Markt oder Verzögerung beim WS sein
    (
        "Kapazitätsprobleme",
        (
            'Kapazitätsengpässe bei der Herstellung',
            'Erhöhter Mehrbedarf',
        ),
    ),
    (
        "Verzögerung",
        (
            'Verzögerung bei der Auslieferung',
            'Verzögerung bei der Herstellung',
            'Verzögerung in der Packmittelbeschaffung',
        ),
    ),
    (
        "Qualitätsprobleme & Sonstiges",
        (
            'Qualitätsprobleme des Fertigproduktes',
            'Rückruf von Chargen am Markt',
            'Qualitätsprobleme der Bulkware',
            'Verzögerung in der Freigabe des Fertigproduktes',
            'Auslieferungsstopp aufgrund eines Qualitätsmangels',
            'GMP-Inspektionsverfahren für den Hersteller noch nicht abgeschlossen',
            'Hersteller nicht GMP-konform',
            'Qualitätsprobleme bei der Herstellung',
            'Untersuchungen beim Hersteller aufgrund von GMP-Problemen',
            'Verzögerung in der Auslieferung aufgrund der Umsetzung der Fälschungsrichtlinie',
            "Sonstiger",
        ),
    ),
    # ("Sonstiger", ('Sonstiger',)),
)

GRUND_MAP = {g: k for (k, gs) in GRUND_DEFINITION for g in gs}


def plt_med_missing_reason():
    display("Grundkategorien:", GRUND_DEFINITION)

    query = "Status not in('teilweise verfügbar', 'verfügbar')"
    if False:
        fig, ax = plt.subplots(figsize=(5, 8))
        fig.suptitle(query + "\nje Wirkstoff (Mehrfachzählung bei mehreren Wirkstoffen)", x=0)
        ax = (
            sx.query(query)
            .explode("Wirkstoffe")
            .value_counts("Wirkstoffe", ascending=True, dropna=False)
            .where(lambda x: x > 2)
            .dropna()
            .plot.barh()
        )
        ax.bar_label(ax.containers[0])
    pdata = sx.query(query).explode("Wirkstoffe")
    pdata["Grund"] = pdata["Grund"].replace(GRUND_MAP)
    anz = 5
    topws = pdata.value_counts("Wirkstoffe", dropna=False).where(lambda x: x >= anz).dropna().index
    pdata = pdata.loc[pdata["Wirkstoffe"].isin(topws)]
    pdata = pdata.sort_values(by="Wirkstoffe", key=lambda ks: ks.map(topws.get_loc))
    fig, ax = plt.subplots(layout="constrained")
    ax = sns.histplot(
        pdata,
        y="Wirkstoffe",
        hue="Grund",
        multiple="stack",
        # dodge=False,
        stat="count",
        discrete=True,
    )
    loc = matplotlib.ticker.AutoLocator()
    loc.set_params(integer=True)
    ax.xaxis.set_major_locator(loc)

    sns.move_legend(
        ax,
        "lower center",
        bbox_to_anchor=(0.5, 1.1),
        ncol=5,
        title=None,
        frameon=False,
    )
    fig = ax.figure
    fig.suptitle(
        f"Mangel-Medikamente je Wirkstoff (Mehrfachzählung bei mehreren) und Grund (nur Anzahl >= {anz})",
        # y=1.02
    )
    ax.set_title(
        MED_LONGSTAMP
        + "\nArzneimittelspezialitäten, auf der Vertriebseinschränkungs-Liste des BASG, bei denen keine Packung voll verfügbar ist",
        # y=1.06,
    )
    ax.set_xlabel("Anzahl verschiedener Medikamente (Mehrfachzählung bei mehreren) ")
    # fig.legend(ax.get_legend())
    # fig.legend(ncol=3, bbox_to_anchor=(0.5, 1), frame_on=False)


if plt_experimental:
    plt_med_missing_reason()

In [ ]:
if False:
    sx0 = sx.copy()
    sx0["Wirkstoffe"] = sx0["Wirkstoffe"].str.join(", ")
    sx0.to_csv(
        util.COLLECTROOT / "medshort/VertriebseinschraenkungenASP_agg.csv", sep=";", encoding="utf-8-sig", index=False
    )

# SARI-Krankenhaus-Monitoring

* Daten von: Österreichischer Dachverband der Sozialversicherungsträger
* Daten-URL: <https://sari-dashboard.at/>
* Daten-Aktualisierung: Wöchentlich (Dienstags)
* Mehr Daten-Infos: [OTS des Gesundheitsministeriums zur Einführung](https://www.ots.at/presseaussendung/OTS_20230810_OTS0005/fruehwarnsystem-zur-ueberwachung-schwerer-atemwegsinfektionen-in-krankenhaeusern-startet)

In [ ]:
covdata = reload(covdata)
_sari_ren_common = {"STATION": "Station", "AUFNAHMEN": "Aufnahmen", "INFLUENZA": "Influenza", "SONSTIGE": "Sonstige"}


def normalize_sari_common(sari: pd.DataFrame, blcol: str) -> pd.DataFrame:
    sari = sari.copy()
    sari["Datum"] = pd.to_datetime(sari["KW"] + " 1", format="%V. KW %G %u")  # .dt.to_period("W")
    last_mdate = None
    frames = []
    for _, grp in sari.groupby("FileDate", as_index=False, sort=False):
        mdate = grp["Datum"].max()
        if last_mdate is None or last_mdate < mdate:
            last_mdate = mdate
            frames.append(grp)
    sari = pd.concat(frames)

    sari["Bundesland"] = sari[blcol].map(
        {
            "BGL": "Burgenland",
            "KTN": "Kärnten",
            "NÖ": "Niederösterreich",
            "OÖ": "Oberösterreich",
            "SBG": "Salzburg",
            "ST": "Steiermark",
            "T": "Tirol",
            "V": "Vorarlberg",
            "W": "Wien",
        }
    )
    assert not pd.isna(sari["Bundesland"]).any()

    sari.rename(columns=_sari_ren_common, errors="raise", inplace=True)
    return sari


def load_sari_kh():
    sari = covdata.loadall_csv("sari/SARI_Region_Krankenanstalt_v202307_*_*.csv", min_distance=timedelta(2))
    sari = normalize_sari_common(sari, "BUNDESLAND")
    sari.set_index(["FileDate", "Datum", "Bundesland", "Station"], inplace=True, verify_integrity=True)
    sari.sort_index(inplace=True)
    return sari


def load_sari_patients():
    sari = covdata.loadall_csv("sari/SARI_Wohnregion_Patient_v202307_*_*.csv", min_distance=timedelta(2))
    sari = normalize_sari_common(sari, "WOHNORT")

    sari.rename(
        columns={
            "GESCHLECHT": "Geschlecht",
            "ALTERSGRUPPE": "Altersgruppe",
            "BEV_ZAHL": "AnzEinwohner",
        },
        errors="raise",
        inplace=True,
    )

    sari["AgeFrom"] = sari["Altersgruppe"].map(
        {"0 - 4": 0, "5 - 14": 5, "15 - 29": 15, "30 - 44": 30, "45 - 59": 45, "60 - 69": 60, "70 - 79": 70, "80+": 80}
    )
    assert np.isfinite(sari["AgeFrom"]).all()
    sari.set_index(
        ["FileDate", "Datum", "Bundesland", "Station", "AgeFrom", "Geschlecht"],
        inplace=True,
        verify_integrity=True,
    )
    sari.sort_index(inplace=True)
    sari = cov.sum_rows(
        sari,
        "Station",
        "alle",
        {k: "sum" if k not in ("AnzEinwohner", "Altersgruppe") else "first" for k in sari.columns},
    ).reset_index("Station", drop=True)

    return sari

In [ ]:
sari_pati_hist = load_sari_patients()
sari_kh_hist = load_sari_kh()
sari_pati = sari_pati_hist.xs(sari_pati_hist.index.unique("FileDate").max(), level="FileDate", drop_level=False)
SARI_DT = sari_pati.index.get_level_values("FileDate").max()
SARI_STAND = "Stand " + SARI_DT.strftime("%d.%m.%y")
DS_SARI = "Statistische Daten des Dachverbandes der Sozialversicherungsträger"

In [ ]:
covdata = reload(covdata)
cov = reload(cov)


def plt_sari_pati_mdiff(sari, place, pltcol="Aufnahmen", *, estimate=False, splitcol="Bundesland", showsum=True):
    name = pltcol if pltcol == "Aufnahmen" else pltcol + "-Aufnahmen"
    fdates = sari.index.unique("FileDate")
    icols = pltcol.split("+")
    if len(icols) > 1:
        sari = sari.copy()
        sari[pltcol] = sari[icols].sum(axis="columns")
    agg = {pltcol: "sum"}
    sari = sari.groupby([splitcol, "FileDate", "Datum"], sort=False).agg(agg)
    if showsum:
        sumname = "Österreich" if splitcol == "Bundesland" else "Gesamt"
        sari = cov.add_sum_rows(sari, splitcol, sumname, {pltcol: "sum"})
        if False:
            sariat0 = sari.xs(sumname)
            olddate = sariat0.index.unique("FileDate").min()
            sariat0old = sariat0.xs(olddate)
            fig, ax = plt.subplots()
            bs = ax.bar(sariat0old.index, sariat0old[pltcol], color="darkgreen", width=6)
            plt.setp(bs[-2:], color="grey")
            cov.set_date_opts(ax)
            ax.set_xlabel("Woche ab")
            fig.suptitle(f"Österreich: Krankenhaus-{name}, Stand " + olddate.strftime("%x"), y=0.93)
            ax.set_ylabel("Aufnahmen")
    else:
        sumname = None
    # display(sari)
    if True:
        fig, axs = cov.plt_mdiff(
            sari.xs(fdates[-1], level="FileDate", drop_level=False).reset_index(),
            sari.xs(fdates[-2], level="FileDate", drop_level=False).reset_index(),
            splitcol,
            vcol=pltcol,
            rwidth=1,
            ndays=300,
            logview=False,
            name=name.replace("-Aufnahmen", "-Aufn."),
            color="grey",
            sharey=False,
            weekly=True,
            multiday=False,
        )
        for ax, (bl, sari0) in zip(axs.flat, sari.groupby(splitcol, sort=False), strict=False):
            if estimate:
                esti = covdata.calc_esti(sari0[pltcol])
                sariat0_last = esti.change_single.loc[
                    esti.change_single["FileDate"] == esti.change_single["FileDate"].max()
                ]
                sariat0_last = sariat0_last.iloc[: esti.esti_len]
                # display(pltcol, bl, sariat0_last, esti.change_agg_inner_cum)
                ax.fill_between(
                    sariat0_last.index.get_level_values("Datum"),
                    sariat0_last[pltcol] * esti.change_compl["min"],
                    sariat0_last[pltcol] * esti.change_compl["max"],
                    color="C0",
                    alpha=0.15,
                )
                ax.set_ylim(
                    top=max(ax.get_ylim()[1], (sariat0_last[pltcol] * esti.change_compl["median"]).max() * 1.03)
                )
            border = matplotlib.dates.date2num(sari.index.get_level_values("Datum")[-1]) - 7 * 2 - 3.5
            h = ax.axvspan(
                border, ax.get_xlim()[1], color="yellow", alpha=0.2, label="Offiziell unvollständiger Bereich", zorder=0
            )
            ax.axvline(border, color="k", ls="--", lw=1)
        fig.suptitle(f"SARI-Krankenhaus-{name} nach {place}", y=0.95)
        cov.stampit(fig, DS_SARI)

    if not estimate or not showsum:
        return

    sariat = sari.xs(sumname, level=splitcol)

    esti = covdata.calc_esti(sariat[pltcol], local_esti=plt_experimental)
    # change_cum_r = change_cum_r.reindex(change_cum_r.index.revere

    fig, ax = plt.subplots(layout="constrained")
    fig.suptitle(f"Österreich: Gemeldete & Geschätzte SARI-{name} (aus Daten nach {place})")
    ax.set_title(
        "Schätzung basiert derzeit nur auf wenigen Aktualisierungszyklen, Schwankungsbreite bzw. Validität schlecht einschätzbar"
    )

    sariat = esti.change_single

    sari_last_df = sariat.loc[sariat["FileDate"] == sariat["FileDate"].max()]
    sari_last = sari_last_df[pltcol]
    ax.plot(sari_last.reset_index("i_age", drop=True), marker="o", label="Aktueller Stand (bisher gemeldet)")
    sari_prev = sariat.loc[sariat["FileDate"] == sariat["FileDate"].unique()[-2], pltcol]
    ax.plot(sari_prev.reset_index("i_age", drop=True), marker=".", ls="--", label="Vorheriger Stand", color="C0")
    sari_last_esti = (
        (sari_last.mul(esti.change_compl["median"], fill_value=1)).reset_index("i_age", drop=True).sort_index()
    )

    for fd, sariat0 in esti.change_single.sort_values("FileDate", kind="stable", ascending=False).groupby(
        "FileDate", as_index=False, sort=False
    ):
        sariat0 = sariat0.reset_index("i_age").sort_index()
        params = dict(color="C1", marker="s", lw=0.5, alpha=0.7, markersize=4)
        if fd == sariat["FileDate"].max():
            params |= dict(label="Schätzung mit aktuellsten Quoten")
            locname = "Konservative Schätzung lt Beesley LJ et al. 2022 Eq(8)"
        else:
            locname = None
        ax.plot(
            sariat0[pltcol] * sariat0["change_rel_cum"].loc[sariat0.index.max() - esti.esti_len * timedelta(7) :],
            **params,
        )
        if plt_experimental:
            ax.plot(
                sariat0[pltcol] * sariat0["change_loc"].loc[sariat0.index.max() - esti.esti_len * timedelta(7) :],
                color="k",
                marker="x",
                label=locname,
            )

    esti_upper = (sari_last * esti.change_compl["max"]).reset_index("i_age", drop=True)
    ax.fill_between(
        sari_last.index.get_level_values("Datum"),
        (sari_last * esti.change_compl["min"]).reset_index("i_age", drop=True),
        esti_upper,
        color="C1",
        alpha=0.15,
        label="Beobachtete Schwankungsbreite",
        zorder=1,
    )

    esti_med = (
        (sari_last * esti.change_compl["median"]).reset_index("i_age", drop=True).sort_index().tail(esti.esti_len)
    )
    ax.plot(
        esti_med,
        color="C3",
        marker="s",
        label="Schätzung mit Medianquoten v2",
    )
    # display(esti_med)

    cov.set_date_opts(ax)
    ax.legend(loc="upper left")
    ax.set_ylim(bottom=0)
    ax.set_ylabel(name)
    ax.set_xlim(*ax.get_xlim())
    ds_esti = DS_SARI
    ax.set_facecolor((0, 0, 0, 0))
    stand_esti = f"SARI: {SARI_STAND}"
    if pltcol == "COVID":
        nn = natmon_norm_interp[(matplotlib.dates.date2num(natmon_norm_interp.index) >= ax.get_xlim()[0] - 1)]
        offset = 0
        nn.index += timedelta(offset)
        ax2 = ax.twinx()
        ax2.set_zorder(-1)
        compl_border = natmon_norm[natmon_norm.index <= ww_completeuntil].index[-1] + timedelta(offset)
        ax2.plot(
            nn[nn.index <= compl_border],
            color="C4",
            label="COVID-Abwasserlast (nicht in Schätzung verwendet)",
        )
        ax2.plot(nn[nn.index >= compl_border], color="C4", alpha=0.7, lw=0.5)
        ax2.legend(loc="lower right")
        ax2.set_ylim(bottom=0)
        ax2.set_ylabel("Abwasser-Virus-Genkopien pro Tag / EW * 10⁶")
        ax2.grid(False)
        ds_esti += ", " + DS_WW
        stand_esti += f", Abwasser: {NATMON_STAND}"
        maxdate = esti_med.idxmax()
        maxval = esti_med.loc[maxdate]
        ww_maxval_align = natmon_norm.loc[
            (natmon_norm.index <= maxdate + timedelta(0)) & (natmon_norm.index >= maxdate - timedelta(20))
        ].max()
        conv_fac = maxval / ww_maxval_align
        ww_maxval = max(natmon_norm.max(), esti_upper.loc[maxdate] / conv_fac)
        maxval_adj = max(maxval, ww_maxval * conv_fac)
        # display(f"{maxdate=} {maxval=} {ww_maxval_align=} {conv_fac=} {ww_maxval=} {maxval_adj=}")
        ax.set_ylim(0, maxval_adj * 1.02)
        ax2.set_ylim(0, ww_maxval * 1.02)

    cov.stampit(fig, ds_esti + ", " + stand_esti)

    debug_esti = True
    if debug_esti:
        # display(esti.change_single)
        fig, ax = plt.subplots(layout="constrained")
        # cov.set_logscale(ax)
        # cov.set_percent_opts(ax, decimals=3)
        fig.suptitle(f"Korrekturfaktor je Datenalter zu: Österreich: SARI-{name} (aus Daten nach {place}) (log)")
        ax.set_ylabel("Korrekturfaktor")
        ax.set_xlabel("Datenalter (Wochen bzw. Aktualisierungszyklen)")
        i_age = esti.change_compl.index.get_level_values("i_age")
        ax.fill_between(i_age, esti.change_compl["min"], esti.change_compl["max"], alpha=0.3, color="C1")
        ax.plot(i_age, esti.change_compl["median"], marker="o")

    fig, ax = plt_growfig(sari_last_esti, f"Österreich: Geschätzte SARI-{name}", showlast=True)
    cov.stampit(fig, DS_SARI + ", " + SARI_STAND)

    # display(sariat)


def plt_growfig(data, ttl, showlast=True):
    fig, ax = plt.subplots()
    color = sns.color_palette(n_colors=1)[0]

    def gtox(s):
        s = s.copy()
        s[s < 1] = -(1 / s) + 1
        s[s >= 1] -= 1
        return s

    gchange = data / data.shift()
    change = gtox(gchange)
    ndays = len(data)
    ax.axhline(0, color="k", lw=1, zorder=1)
    bar_args = dict(
        zorder=2,
        lw=0,
        aa=True,
        snap=False,
        # alpha=0.3,
        width=5,
    )
    ax.bar(
        change.index,
        height=change.where(change < 0),
        color=color,
        edgecolor=color,
        label="Fallend",
        **bar_args,
    )
    ax.bar(
        change.index,
        height=change.where(change >= 0),
        color="C3",
        edgecolor="C3",
        label="Steigend",
        **bar_args,
    )
    valdates = data.loc[data.first_valid_index() :].index
    alwaysshow = int(-len(valdates) * 0.2)
    ax.set_ylim(
        top=min(
            3, max(min(ax.get_ylim()[1], 1.5), change.iloc[alwaysshow:].quantile(0.99, interpolation="higher") * 1.05)
        ),
        bottom=max(
            -3, min(max(ax.get_ylim()[0], -1.5), change.iloc[alwaysshow:].quantile(0.01, interpolation="lower") * 1.05)
        ),
    )

    def fmtx(v, _):
        return f"×{v + 1:.2f}" if v >= 0 else f"÷{-v + 1:.2f}"

    if showlast:
        ax.axhline(change.iloc[-1], color="C0" if change.iloc[-1] < 0 else "C3", lw=1)
        ax.annotate(
            fmtx(change.iloc[-1], None)
            + f"\n= {gchange.iloc[-1] - 1:+.1%}".replace("-", "−")
            + (("\n×2 in " + format(doubling_time(gchange.iloc[-1]) * 7, ".0f") + " T") if change.iloc[-1] > 0 else ""),
            (change.index[-1], change.iloc[-1]),
            xytext=(10, 0),
            va="bottom",
            textcoords="offset points",
        )
    cov.set_date_opts(ax)
    ax.set_xlim(left=valdates[0], right=valdates[-1] + timedelta(6) + (valdates[-1] - valdates[0]) * 0.01)
    fig.legend(loc="upper center", ncol=4, frameon=False, bbox_to_anchor=(0.5, 0.94))
    fig.suptitle(f"{ttl}: Änderung gegenüber 7 Tage vorher", y=0.96)
    # ax.yaxis.set_major_formatter(matplotlib.ticker.FuncFormatter(lambda v, _: f"×{v:.1f}"))
    ax.yaxis.set_major_formatter(matplotlib.ticker.FuncFormatter(fmtx))
    fig.autofmt_xdate()
    return fig, ax


def doubling_time(g):
    return np.log(2) / np.log(g)


# plt_sari_pati_mdiff(sari_kh_hist.query("FileDate < '2023-08-19T00:00Z'"), "Krankenhausstandort", "COVID", estimate=False)
plt_sari_pati_mdiff(sari_kh_hist, "Krankenhausstandort", estimate=True)
if True:
    plt_sari_pati_mdiff(sari_kh_hist, "Krankenhausstandort", "COVID", estimate=True)
    plt_sari_pati_mdiff(sari_kh_hist, "Krankenhausstandort", "Sonstige", estimate=True)
    plt_sari_pati_mdiff(sari_kh_hist, "Krankenhausstandort", "Influenza+RSV", estimate=False)
    plt_sari_pati_mdiff(
        cov.sum_rows(sari_pati_hist.set_index("Altersgruppe", append=True), "Bundesland", "Österreich"),
        "Altersgruppe",
        estimate=True,
        splitcol="Altersgruppe",
        showsum=False,
    )
    plt_sari_pati_mdiff(
        cov.sum_rows(sari_pati_hist.set_index("Altersgruppe", append=True), "Bundesland", "Österreich"),
        # sari_pati_hist.set_index("Altersgruppe", append=True).query("Bundesland == 'Salzburg'"),
        "Altersgruppe",
        "COVID",
        estimate=True,
        splitcol="Altersgruppe",
        showsum=False,
    )
    # plt_sari_pati_mdiff(sari_kh_hist, "Krankenhausstandort", "RSV", estimate=True)
    # plt_sari_pati_mdiff(sari_kh_hist, "Krankenhausstandort", "Sonstige", estimate=True)

In [ ]:
def plt_sari_ag(sari_pati=sari_pati, normalize=True, diag="Aufnahmen"):
    sari_pati = sari_pati.groupby(["Datum", "AgeFrom", "Bundesland"]).agg(
        {
            "AnzEinwohner": "sum",
            "Aufnahmen": "sum",
            "COVID": "sum",
            "RSV": "sum",
            "Influenza": "sum",
            "Sonstige": "sum",
            "Altersgruppe": "first",
        }
    )
    sari_pati = cov.add_sum_rows(
        sari_pati, "Bundesland", "Österreich", {diag: "sum", "AnzEinwohner": "sum", "Altersgruppe": "first"}
    ).reset_index()
    fig, axs = plt.subplots(figsize=(10, 9), nrows=4, ncols=2, sharex=True, sharey=False, layout="constrained")
    # fig.subplots_adjust(wspace=0.3, hspace=0.17)
    # print(sari_pati["Altersgruppe"].unique())
    for selag, ax in zip(sari_pati["Altersgruppe"].unique(), axs.flat, strict=True):
        pdata = sari_pati[sari_pati["Altersgruppe"] == selag]
        pdata = (
            pdata.groupby(["Datum", "Bundesland"])[
                [
                    diag,
                    "AnzEinwohner",
                ]
            ]
            .sum()
            .reset_index()
        )
        yname = diag
        if "Aufnahmen" not in diag:
            yname += "-Aufnahmen"
        if normalize:
            yname += " / 100.000 EW"
        pdata[yname] = pdata[diag]
        if normalize:
            pdata[yname] = pdata[yname] / pdata["AnzEinwohner"] * 100_000
        pdata["prelim"] = pdata["Datum"] >= sari_pati["Datum"].max() - timedelta(21)
        sns.lineplot(
            pdata.where(pdata["prelim"]),
            x="Datum",
            y=yname,
            hue="Bundesland",
            marker="P",
            ls="--",
            legend=False,
            ax=ax,
            size="AnzEinwohner",
        )
        pdata["prelim"] = pdata["Datum"] > sari_pati["Datum"].max() - timedelta(21)
        sns.lineplot(
            pdata.where(~pdata["prelim"]),
            x="Datum",
            y=yname,
            hue="Bundesland",
            marker="o",
            ax=ax,
            legend=ax is axs.flat[0],
            size="AnzEinwohner",
        )
        if ax is axs.flat[0]:
            artists, labels = ax.get_legend_handles_labels()
            bev_idx = labels.index("AnzEinwohner")
            artists, labels = artists[1:bev_idx], labels[1:bev_idx]
            fig.legend(artists, labels, ncol=5, frameon=False, bbox_to_anchor=(0.5, 1), loc="lower center")
            ax.get_legend().remove()
        if ax not in axs.T[0]:
            ax.set_ylabel(None)
        if ax in axs[-1]:
            cov.set_date_opts(ax)
        fmt = matplotlib.ticker.ScalarFormatter(useOffset=False)
        fmt.set_scientific(False)
        ax.yaxis.set_major_formatter(fmt)
        ax.set_title(selag)
        ax.set_ylim(bottom=0)
        if not normalize:
            ax.yaxis.set_major_locator(matplotlib.ticker.MaxNLocator('auto', integer=True))
        # ax.axvline(sari_pati["Datum"].max() - timedelta(14), color="k")
    axs.flat[-1].set_xlim(left=sari_pati["Datum"].min())

    # axs.flat[-1].set_xlabel("Kalenderwoche ab")
    fig.suptitle(f"Österreich: SARI-Krankenhaus-{yname} je Altersgruppe und Bundesland, {SARI_STAND}", y=1.08)
    cov.stampit(fig, DS_SARI)
    plt.setp(
        chain.from_iterable(ax.get_xticklabels() for ax in axs.flat), rotation=40, ha='right', rotation_mode='anchor'
    )


# plt_sari_ag(normalize=False)
# plt_sari_ag(normalize=True, diag="COVID")
# plt_sari_ag(normalize=False, diag="COVID")

In [ ]:
def plt_sari_ag_one(sari_pati=sari_pati, normalize=True, diag="Aufnahmen", extratitle="", logscale=False):
    sari_pati = sari_pati.groupby(["Datum", "AgeFrom", "Bundesland"]).agg(
        {
            "AnzEinwohner": "sum",
            "Aufnahmen": "sum",
            "COVID": "sum",
            "RSV": "sum",
            "Influenza": "sum",
            "Sonstige": "sum",
            "Altersgruppe": "first",
        }
    )
    sari_pati = cov.sum_rows(
        sari_pati, "Bundesland", "Österreich", {diag: "sum", "Altersgruppe": "first", "AnzEinwohner": "sum"}
    )
    sari_pati = sari_pati.reset_index()
    # display(sari_pati)
    fig, ax = plt.subplots(layout="constrained")
    # fig.subplots_adjust(wspace=0.3, hspace=0.17)
    yname = diag
    if "Aufnahmen" not in diag:
        yname += "-Aufnahmen"
    if normalize:
        yname += " / 100.000 EW"
    pdata = sari_pati
    pdata[yname] = pdata[diag]
    if normalize:
        pdata[yname] = pdata[yname] / pdata["AnzEinwohner"] * 100_000
    pdata["prelim"] = pdata["Datum"] >= sari_pati["Datum"].max() - timedelta(21)
    # display(pdata)
    palette = sns.color_palette(n_colors=pdata["Altersgruppe"].nunique())
    sns.lineplot(
        pdata.where(pdata["prelim"]),
        x="Datum",
        y=yname,
        hue="Altersgruppe",
        marker="P",
        ls="--",
        legend=False,
        ax=ax,
        palette=palette,
    )
    pdata["prelim"] = pdata["Datum"] > sari_pati["Datum"].max() - timedelta(21)
    # display(pdata.where(~pdata["prelim"]))
    sns.lineplot(
        pdata.where(~pdata["prelim"]),
        x="Datum",
        y=yname,
        hue="Altersgruppe",
        marker="o",
        ax=ax,
        palette=palette,
    )
    fig.legend(ncol=5, frameon=False, bbox_to_anchor=(0.5, 0.98), loc="lower center")
    ax.get_legend().remove()
    cov.set_date_opts(ax)
    fmt = matplotlib.ticker.ScalarFormatter(useOffset=False)
    fmt.set_scientific(False)
    ax.yaxis.set_major_formatter(fmt)
    # ax.set_title(selag)
    if logscale:
        cov.set_logscale(ax)
    else:
        ax.set_ylim(bottom=0)
    if not normalize:
        ax.yaxis.set_major_locator(matplotlib.ticker.MaxNLocator('auto', integer=True))
    # ax.axvline(sari_pati["Datum"].max() - timedelta(14), color="k")
    ax.set_xlim(left=sari_pati["Datum"].min())

    # axs.flat[-1].set_xlabel("Kalenderwoche ab")
    fig.suptitle(f"Österreich: SARI-Krankenhaus-{yname} je Altersgruppe{extratitle}, {SARI_STAND}", y=1.1)
    cov.stampit(fig, DS_SARI)
    cov.labelend2(ax, pdata, yname, "Altersgruppe", shorten=lambda n: n, colorize=palette)


plt_sari_ag_one(sari_pati.query("AgeFrom < 80"), extratitle=", nur <80")
plt_sari_ag_one(sari_pati.query("AgeFrom < 80"), diag="COVID", extratitle=", nur <80")
plt_sari_ag_one(sari_pati, diag="COVID", extratitle=", logarithmisch", logscale=True)
# cov.set_logscale(plt.gca())

Tabellarische Darstellung der Krankenhaus-Aufnahmen, aus Daten nach Krankenhausstandort (Gesamt-Österreich, alle Stationen; Erste Tabelle COVID, zweite alle am SARI-Dashboard erfassten Diagnosen):

In [ ]:
def tabulate_sari():
    for cname in ("COVID", "Aufnahmen"):
        display(
            sari_kh_hist.assign(Stand=sari_kh_hist.index.get_level_values("FileDate").strftime("%m-%d"))
            .loc[
                lambda df: df.index.get_level_values("FileDate") >= df.index.get_level_values("FileDate").unique()[-15]
            ]
            .groupby(["Datum", "Stand"])[[cname]]
            .sum()
            .reset_index()
            .pivot(index="Datum", columns="Stand")
            .tail(60)
            .style.format(precision=0, na_rep="-")
            .format_index(formatter=lambda dt: dt.strftime("%d.%m %yW%V"))
        )


tabulate_sari()

# Krankenstand

* Daten von: Österreichischer Dachverband der Sozialversicherungsträger
* Daten-URL: <https://www.sozialversicherung.at/cdscontent/?contentid=10007.853001&portal=svportal>
* Daten-Aktualisierung: Monatlich, ca. ab 25.
* [Mehr Daten-Infos](https://github.com/zeitferne/covidat-data/blob/main/docs/sozialversicherung-monatsberichte.md)

In [ ]:
ks = pd.read_csv(
    util.COLLECTROOT / "sozialversicherung-monatsberichte/ks_all.csv",
    sep=";",
    encoding="utf-8",
    dtype={"date": pd.PeriodDtype("M")},
    index_col=["date", "insurer", "employment"],
)

KS_ANCHOR_MONTH = 6
KS_ANCHOR_MONTH_NAME = "JUN"

ks["jun_year"] = ks.index.get_level_values("date").asfreq("A-" + KS_ANCHOR_MONTH_NAME).year - 1
if KS_ANCHOR_MONTH == 12:
    ks["jun_year_l"] = (ks["jun_year"] + 1).astype(str)
else:
    ks["jun_year_l"] = ks["jun_year"].astype(str) + "/" + ((ks["jun_year"] + 1) % 100).astype(str)
ks["jun_month"] = (ks.index.get_level_values("date").month - 1 + (12 - KS_ANCHOR_MONTH)) % 12 + 1
ks["cases_n"] = ks["cases"] / ks.index.get_level_values("date").days_in_month * 30
ks["m_start"] = ks.index.get_level_values("date").start_time

# ks["qyear"] = ks["qdate"].dt.qyear
ks = cov.add_sum_rows(
    ks,
    "employment",
    "Insgesamt",
    {
        "insured": "sum",
        "cases": "sum",
        "cases_n": "sum",
        "active_end": "sum",
        "jun_year": "first",
        "jun_year_l": "first",
        "jun_month": "first",
        "m_start": "first",
    },
)

ks["active_end_pp"] = ks["active_end"] / ks["insured"]
ks["cases_pp_n30"] = ks["cases_n"] / ks["insured"]
ks["cases_pp_raw"] = ks["cases"] / ks["insured"]
# ks_s["cases_pp_n30"] = ks_s["cases_n"] / ks_s["insured"]
# ks_s["active_end_pp"] = ks_s["active_end"] / ks_s["insured"]
# ks["m_start"] = ks0["date"].dt.start_time
# display(ks_s.query("insurer == 'Insgesamt' and employment != 'angest'")[["jun_year", "month"]])

In [ ]:
def rotate(l, n):
    return l[n:] + l[:n]


def plt_ks_yoy(y, bl, empl="Insgesamt"):
    whatlabel = (
        "Krankenstandfälle/Monat (30-Tage-normiert)"
        if y == "cases_pp_n30"
        else "Laufende Krankenstände am Monatsende"
        if y == "active_end_pp"
        else None
    )
    empllabel = "" if empl == "Insgesamt" else f" (nur {empl})"
    fig, ax = plt.subplots()
    # display(ks.dtypes)
    # ax.set_xlabels(["])
    blq = "Insgesamt" if bl == 'Österreich' else bl
    pdata = ks.query(f"insurer == '{blq}' and employment == '{empl}'").sort_index().reset_index()
    pal = sns.color_palette("tab10", n_colors=pdata["jun_year_l"].nunique())[::-1]
    markersize = 9
    lw = 3
    pltargs = dict(x="jun_month", y=y, hue="jun_year_l", palette=pal, sort=False, markersize=markersize)
    sns.lineplot(
        pdata,
        # lw=3,
        # size=pdata["jun_year_l"].map(lambda y: 3 if y == pdata.iloc[0]["jun_year_l"] else 2),
        # sizes=(2, 4),
        marker="o" if y != "active_end_pp" else None,
        lw=lw,
        **pltargs,
    )
    extrahandles = []
    if y == "active_end_pp":
        wday = pdata["date"].dt.end_time.dt.dayofweek
        is_weekend = wday > 4
        is_monday = wday == 0
        pd0 = pdata.copy()
        pd0.loc[is_monday | is_weekend, y] = np.nan
        sns.lineplot(pd0, marker="o", lw=0, **pltargs, legend=False)
        pd0 = pdata.copy()
        pd0.loc[~is_monday, y] = np.nan
        sns.lineplot(pd0, marker="X", lw=0, **pltargs, legend=False, mew=0)
        extrahandles += [
            matplotlib.lines.Line2D([], [], color="k", lw=lw, markersize=markersize, label="Mo", marker="X"),
            matplotlib.lines.Line2D([], [], color="k", lw=lw, markersize=markersize, label="Di-Fr", marker="o"),
            matplotlib.lines.Line2D([], [], color="k", lw=lw, markersize=markersize, label="Sa/So"),
        ]

    cov.set_percent_opts(ax)
    if KS_ANCHOR_MONTH != 12:
        ax.axvline(12 - KS_ANCHOR_MONTH + 1 - 0.5, color="k", ls="--", lw=1, zorder=1)
    ax.set_xticks(np.arange(1, 12 + 1))
    ax.set_xticklabels(
        rotate(["Jän", "Feb", "Mär", "Apr", "Mai", "Jun", "Jul", "Aug", "Sep", "Okt", "Nov", "Dez"], KS_ANCHOR_MONTH)
    )
    ax.set_ylim(bottom=0)
    ax.set_xlabel("Monat")
    # ax.set_ylabel("Anteil der Versicherten (30-Tage-normiert)")
    ax.set_ylabel("Fälle/Versicherte")
    # fig.suptitle("Aktive Krankenstandfälle am Monatsende als Anteil der Versicherten", y=0.93)
    fig.suptitle(f"{bl}{empllabel}: {whatlabel} als Anteil der Versicherten", y=0.93)

    lastlinedata = pdata[pdata["jun_year"] == pdata["jun_year"].max()]
    last_over_jan = lastlinedata["date"].iloc[0].year != lastlinedata["date"].iloc[-1].year
    kwargs = dict(ax=ax, mms=pdata, ycol=y, cats="jun_year", x="jun_month", colorize=pal)
    cov.labelend2(
        **kwargs,
        shorten=lambda x: str(
            (x + 1 if x != lastlinedata["jun_year"].iat[-1] or KS_ANCHOR_MONTH == 12 or last_over_jan else x) % 100
        ),
    )
    cov.labelend2(
        **kwargs,
        ends=(True, False),
        shorten=lambda x: str((x + 1) % 100) if x == 2016 or KS_ANCHOR_MONTH == 12 else str(x % 100),
    )

    ax.yaxis.set_major_locator(matplotlib.ticker.MultipleLocator(0.02))
    cov.stampit(fig, "Monatsberichte des österr. Dachverband der Sozialversicherungsträger")
    ax.legend(handles=ax.get_legend_handles_labels()[0] + extrahandles, ncol=3, title="Jahr bzw. Saison")


if False:
    for bl in cov.BUNDESLAND_BY_ID.values():
        plt_ks_yoy("cases_pp_n30", bl)

plt_ks_yoy("cases_pp_n30", "Österreich", "Insgesamt")
plt_ks_yoy("cases_pp_n30", "Österreich", "ArbeiterInnen")
plt_ks_yoy("cases_pp_n30", "Österreich", "Angestellte")

Die Krankenstand-Endstände mit Stichtag Monatsletzter, wie im Folgenden Diagramm dargestellt müssen besonders vorsichtig interpretiert werden, wegen sehr starken Feiertags-, Fenstertags- & Wochentagseffekten (Wochentag am Monatsletzten: Siehe Markierung/Legende).

In [ ]:
plt_ks_yoy("active_end_pp", "Österreich")

In [ ]:
def plt_ks_bl_all():
    fig, ax = plt.subplots(layout="constrained")
    ks0 = ks.query("employment == 'Insgesamt'").reset_index()  # .query("employment == 'angest'").reset_index()
    sns.lineplot(
        ks0.query(
            "insurer in ('Vorarlberg', 'Tirol', 'Salzburg', 'Oberösterreich', 'Kärnten', 'Steiermark', 'Niederösterreich', 'Wien', 'Burgenland')"
        ),
        hue="insurer",  # style="employment",
        x="m_start",
        y="cases_pp_n30",
        marker=".",
        mew=0,
    )
    cov.set_percent_opts(ax, decimals=1)
    ax.xaxis.set_major_locator(matplotlib.dates.YearLocator())
    ax.xaxis.set_minor_formatter(matplotlib.dates.DateFormatter("%b"))
    ax.xaxis.set_minor_locator(matplotlib.dates.MonthLocator([1, 4, 7, 10]))
    ax.set_ylim(bottom=0)

    ax.grid(which="minor", lw=0.3)
    # fig.autofmt_xdate()
    ax.tick_params(axis="x", which="major", pad=11)
    # cov.set_logscale(ax)
    ax.legend(ncol=2, loc="lower left")
    fig.suptitle("Krankenstandzugang/Monat (normiert auf 30 Tage) in % der Versicherten")
    ax.set_title("Arbeiter + Angestellte, je Versichernder ÖGK")
    ax.set_xlabel("Monat")
    ax.set_ylabel("Fälle/Versicherte (30-Tage-normiert)")
    ax.set_xlim(ks0["m_start"].min() - timedelta(14), ks.index.get_level_values("date").max().end_time + timedelta(62))
    cov.stampit(fig, "Monatsberichte des österr. Dachverband der Sozialversicherungsträger")


plt_ks_bl_all()

In [ ]:
def plt_emp_ks_all():
    logscale = False
    fig, ax = plt.subplots(layout="constrained")
    if logscale:
        cov.set_logscale(ax)
    fig.suptitle("Krankenstandzugang/Monat (normiert auf 30 Tage) in % der Versicherten")
    ax.set_title("Über alle im Bericht erfassten Träger, je Arbeitsverhältnis")

    pdata = ks.query("insurer == 'Insgesamt'").reset_index()
    pargs = dict(x="m_start", y="cases_pp_n30", estimator=None, marker=".", mew=0, ax=ax)
    ax = sns.lineplot(pdata.query("employment != 'Insgesamt'"), hue="employment", ls="--", **pargs)
    sns.lineplot(
        pdata.query("employment == 'Insgesamt'"), lw=2, markersize=10, color="grey", label="Insgesamt", **pargs
    )
    cov.set_percent_opts(ax, decimals=1)
    ax.xaxis.set_major_locator(matplotlib.dates.YearLocator())
    ax.xaxis.set_minor_formatter(matplotlib.dates.DateFormatter("%b"))
    ax.xaxis.set_minor_locator(matplotlib.dates.MonthLocator([1, 4, 7, 10]))
    ax.grid(which="minor", lw=0.3)
    # fig.autofmt_xdate()
    ax.tick_params(axis="x", which="major", pad=11)
    if not logscale:
        ax.set_ylim(bottom=0)
    ax.set_xlim(pdata["m_start"].min() - timedelta(14), pdata["date"].max().end_time + timedelta(62))

    ax.set_xlabel("Monat")
    ax.set_ylabel("Fälle/Versicherte (30-Tage-normiert)")

    ax.legend()
    cov.stampit(fig, "Monatsberichte des österr. Dachverband der Sozialversicherungsträger")


plt_emp_ks_all()

In [ ]:
if plt_experimental:
    ksogk = ks.xs("Insgesamt", level="employment").query(
        "insurer == 'Österr. Gesundheitskasse' or insurer == 'Gebietskrankenkassen'"
    )  # xs("Insgesamt", level="insurer")
    display(
        ksogk.groupby([ksogk.index.get_level_values("date").year.rename("Jahr"), "insurer"])
        .agg({"cases": "sum", "insured": "mean", "cases_pp_raw": "sum"})
        .rename(
            columns={
                "cases": "Krankenstandfälle",
                "insured": "Im Schnitt versichert (ca.)",
                "cases_pp_raw": "Fälle/Versicherte",
            }
        )
        .style.format(formatter=lambda v: f"{v:,.0f}".replace(",", " "))
        .format(subset="Fälle/Versicherte", formatter=lambda v: f"{v:.2f}")
    )

# Sterberaten

* Daten von: Statistik Austria - data.statistik.gv.at
* Daten-URLs:
  * <https://data.statistik.gv.at/web/meta.jsp?dataset=OGD_gest_kalwo_GEST_KALWOCHE_100> (Grobe Absolutzahlen, derzeit nicht verwendet)
  * <https://data.statistik.gv.at/web/meta.jsp?dataset=OGD_rate_kalwo_GEST_KALWOCHE_STR_100> (ASR)
  * <https://data.statistik.gv.at/web/meta.jsp?dataset=OGD_gest_kalwo_alter_GEST_KALWOCHE_5J_100> (5-Jahres-Gruppen, derzeit nicht verwendet)
* Daten-Aktualisierung: Monatlich, ca. am letzten Donnerstag

In [ ]:
def split_date_state(dasr, stateprefix):
    dasr["date"] = pd.to_datetime(dasr["date"] + "1", format="KALW-%G%V%u")
    dasr["date"].freq = "W"
    dasr["year"] = dasr["date"].dt.isocalendar().year.astype(int)
    dasr["week"] = dasr["date"].dt.isocalendar().week.astype(int)
    dasr["state_id"] = dasr["state_id"].str.removeprefix(stateprefix).astype(int)


def map_state(state_ids):
    return state_ids.map(lambda i: "Österreich" if i == 0 else cov.BUNDESLAND_BY_ID[i])


def sum_rows(df, cname, csval):
    dfsum = df.groupby(level=[n for n in df.index.names if n != cname])[["n"]].sum()
    dfsum[cname] = csval
    dfsum.set_index(cname, append=True, inplace=True, verify_integrity=True)
    return dfsum.reorder_levels(df.index.names)


def add_sum_rows(df, cname, csval):
    return pd.concat([df, sum_rows(df, cname, csval)])


def sum_state_sex(d5y):
    agg = {c: "first" if c in ("year", "week") else "sum" for c in d5y.columns}
    d5y = cov.add_sum_rows(d5y, "state_id", 0, agg)
    d5y = cov.add_sum_rows(d5y, "sex", "mf", agg)
    d5y["state"] = map_state(d5y.index.get_level_values("state_id"))
    return d5y.sort_index()

In [ ]:
dsum_o = pd.read_csv(
    util.DATAROOT / "statat-weekly-deaths/OGD_gest_kalwo_GEST_KALWOCHE_100_latest.csv",
    encoding="ascii",
    sep=";",
    decimal=",",
)

In [ ]:
dsum = dsum_o.rename(
    columns={
        "C-KALWOCHE-0": "date",
        "C-B00-0": "state_id",
        "C-ALTERGR65-0": "age_coarse",
        "C-C11-0": "sex",
        "F-ANZ-1": "n",
    },
    errors="raise",
)
split_date_state(dsum, "B00-")
dsum["sex"] = dsum["sex"].map({"C11-1": "m", "C11-2": "f"})
dsum["age_coarse"] = dsum["age_coarse"].map({"ALTERSGR65-1": 0, "ALTERSGR65-2": 65})
dsum = dsum.set_index([c for c in dsum.columns if c not in ("n", "state")], verify_integrity=True)

# We drop the over/under 65 distinction here
dsum = sum_rows(dsum, "age_coarse", -1).reset_index("age_coarse", drop=True).sort_index()

dsum = sum_state_sex(dsum).reset_index("date")

In [ ]:
dasr_o = pd.read_csv(
    util.DATAROOT / "statat-weekly-deaths/OGD_rate_kalwo_GEST_KALWOCHE_STR_100_latest.csv",
    encoding="ascii",
    sep=";",
    decimal=",",
)

In [ ]:
ESTIMATE_ASR_FROM_RAW = False

dasr = dasr_o.rename(
    columns={"C-KALWOCHE-0": "date", "C-BLWO-0": "state_id", "C-SEXWO-0": "sex", "F-ANZ-1": "n", "F-RATE-1": "rate"},
    errors="raise",
)

split_date_state(dasr, "BLWO-")
# dasr["date"] -= pd.Timedelta(25, 'W')
# dasr["jun_year"] = (dasr["date"] - pd.Timedelta(25, 'W')).dt.isocalendar().year.astype(int)


dasr["sex"] = dasr["sex"].map({"SEXWO-0": "mf", "SEXWO-1": "m", "SEXWO-2": "f"})

dasr.set_index(
    [c for c in dasr.columns if c not in ("n", "state", "date", "rate")], inplace=True, verify_integrity=True
)
if ESTIMATE_ASR_FROM_RAW:
    dasr_lastdate = dasr["date"].max()
    dasr = dsum[["n", "state", "date"]].join(dasr["rate"])
    dasr_pr = dasr
    dasr_pr = dasr_pr[np.isfinite(dasr_pr["rate"]) & np.isfinite(dasr_pr["n"])]
    dasr_pr = dasr_pr.groupby([n for n in dasr_pr.index.names if n not in ("year", "week")]).last()
    dasr["rate"] = dasr["rate"].combine_first(dasr["n"] * (dasr_pr["rate"] / dasr_pr["n"]))
    # display(dasr[~np.isfinite(dasr["rate"])])
    # display(dasr[~np.isfinite(dasr["n"])])
    estimate_count = int(np.round((dasr["date"].max() - dasr_lastdate).days / 7))
    ESTIMATE_DISCLAIMER = f" | Letzte {estimate_count} Wochen grob geschätzt"
else:
    estimate_count = 0
    ESTIMATE_DISCLAIMER = ""
    dasr["state"] = map_state(dasr.index.get_level_values("state_id"))
dasr.set_index("date", append=True, inplace=True)


w3m = dasr.groupby(["state_id", "sex"])["rate"].rolling(5, center=True, min_periods=1).min()
rate_min = w3m.groupby(["state_id", "sex", "week"]).min()

# Need to move "week" before "year" to allow join
dasr = dasr.reorder_levels(["state_id", "sex", "week", "year", "date"])

dasr["prate"] = dasr["rate"] - rate_min
# display(dasr["rate"], rate_min)
dasr.reset_index(inplace=True)

dasr["jun_year"] = dasr["date"].dt.isocalendar().year.astype(int)
is_prejun = dasr["date"].dt.isocalendar().week < 21
dasr.loc[is_prejun, "jun_year"] -= 1
dasr["jun_week"] = dasr["date"].dt.isocalendar().week.astype(int)
dasr.loc[is_prejun, "jun_week"] += (
    dasr["jun_year"]
    .map(lambda y: (pd.Timestamp.fromisocalendar(y + 1, 1, 1) - pd.Timedelta(1, 'W')).isocalendar().week)
    .astype(int)
)

dasr["1ydate"] = calc1ydate("year", dasr)
dasr["jun_1ydate"] = calc1ydate("jun_year", dasr)


def add_period_sums(prefix, idcol):
    dasr.set_index(["state_id", "sex", "week", idcol], inplace=True, verify_integrity=True)
    dasr[prefix + "crate"] = dasr.groupby([idcol, "state_id", "sex"])["rate"].cumsum()
    dasr[prefix + "cprate"] = dasr.groupby([idcol, "state_id", "sex"])["prate"].cumsum()
    dasr.reset_index(inplace=True)


add_period_sums("y", "year")
add_period_sums("jy", "jun_year")
# dasr.set_index(["state_id", "sex", "year", "week"], inplace=True)
# dasr.sort_index(inplace=True)

with open(util.DATAROOT / "statat-weekly-deaths/OGD_rate_kalwo_GEST_KALWOCHE_STR_100_json_latest.json", "rb") as f:
    dasr_meta = json.load(f)
dasr_meta_res = next(r for r in dasr_meta["resources"] if r["name"] == "OGD_rate_kalwo_GEST_KALWOCHE_STR_100")
dasr_moddate = util.parse_statat_date(dasr_meta_res["last_modified"])

# print(dasr["date"].max().strftime("%GW%V %d.%m.%y"))
DASR_LONGSTAMP = (
    dasr_meta["extras"]["license_citation"]
    + " | Darstellung: @zeitferne | Bis "
    + dasr["date"].max().strftime("%GW%V")
    + ", Stand "
    + dasr_moddate.strftime("%d.%m.%y")
)

In [ ]:
if plt_experimental:
    d5y_o = pd.read_csv(
        util.DATAROOT / "statat-weekly-deaths/OGD_gest_kalwo_alter_GEST_KALWOCHE_5J_100_latest.csv",
        encoding="ascii",
        sep=";",
        decimal=",",
    )

In [ ]:
cov = reload(cov)

In [ ]:
def load_d5y():
    d5y = d5y_o.rename(
        columns={
            "C-KALWOCHE-0": "date",
            "C-B00-0": "state_id",
            "C-ALTER5-0": "age",
            "C-C11-0": "sex",
            "F-ANZ-1": "n",
        },
        errors="raise",
    )
    split_date_state(d5y, "B00-")
    d5y["sex"] = d5y["sex"].map({"C11-1": "m", "C11-2": "f", "C11-0": "x"})
    d5y["age"] = (d5y["age"].str.removeprefix("ALTER5-").astype(int) - 1) * 5
    d5y = d5y.set_index([c for c in d5y.columns if c not in ("n", "state")])

    bev5 = cov.bev_to_daily(
        cov.load_bev(ages_age_groups=False),
        d5y.index.unique("date").min().year - 1,
        d5y.index.unique("date").max().year + 1,
        add_at_sum=False,
    )
    dts = bev5.index.get_level_values("Datum")
    bev5 = bev5.loc[
        (dts.weekday == 0) & (dts >= d5y.index.unique("date").min()) & (dts <= d5y.index.unique("date").max())
    ]
    bev5 = bev5.reset_index("Geschlecht")
    bev5["Geschlecht"] = bev5["Geschlecht"].map({"M": "m", "W": "f"})
    bev5.set_index("Geschlecht", append=True, inplace=True, verify_integrity=True)
    bev5.index.set_names(
        (
            {"Datum": "date", "BundeslandID": "state_id", "AgeFrom": "age", "Geschlecht": "sex"}[n]
            for n in bev5.index.names
        ),
        inplace=True,
    )
    # display(bev5)
    bev5["year"] = bev5.index.get_level_values("date").isocalendar().year.astype(int).values
    bev5["week"] = bev5.index.get_level_values("date").isocalendar().week.astype(int).values
    bev5.set_index(["year", "week"], append=True, inplace=True)
    bev5.index = bev5.index.reorder_levels(d5y.index.names)
    d5y = d5y.join(bev5, how="right", validate="one_to_one")
    d5y["n"].fillna(0, inplace=True)
    d5y = sum_state_sex(d5y)
    return d5y


if plt_experimental:
    d5y = load_d5y()

In [ ]:
def plt_d5y(*, bl, agefrom=15, ageto=20):
    rwidth = 53
    normew = True
    sex = "mf"
    blid = tuple(cov.SHORTNAME_BY_BUNDESLAND).index(bl) + 1
    if blid == 10:
        blid = 0
    d5x0 = (
        d5y.xs(blid, level="state_id")
        .xs(sex, level="sex")
        .query(f"age >= {agefrom} and age < {ageto}")
        .groupby(["date"])
        .sum()
    ).tail(1000)

    d5x0 = d5x0["n"] / d5x0["AnzEinwohner"] * 100_000 if normew else d5x0["n"]
    d5x = d5x0.rolling(rwidth).mean()  # .reset_index()
    fig, ax = plt.subplots()
    # ax = d5x.reset_index().query("date == '2023-02-20'").plot(
    #    x="date", y="n", lw=0, marker="X", color="r", markersize=10, legend=False, ax=ax)
    # d5x["date"] = matplotlib.dates.date2num(d5x["date"])
    ax.plot(d5x, marker=".", lw=0.5 if rwidth < 12 else None, mew=0 if rwidth >= 12 else None)

    ax.set_ylabel("Sterbefälle/100.000 EW & KW" if normew else "Sterbefälle/KW")
    rlabel = "" if rwidth == 1 else f", {rwidth}-KW-Schnitt"
    ttl = f"{bl}: {ax.get_ylabel()}, {agefrom}-<{ageto}-Jährige ({sex}){rlabel}"
    ax.figure.suptitle(ttl, y=0.95)
    print(ttl, DASR_LONGSTAMP)
    ax.set_xlabel("Kalenderwoche bzw. Jahr")
    if len(d5x) > 1000:
        ax.xaxis.set_major_locator(matplotlib.dates.YearLocator(2))
        ax.xaxis.set_minor_locator(matplotlib.dates.YearLocator())
        # ax.yaxis.set_major_locator(matplotlib.ticker.MaxNLocator('auto', integer=True))
    elif len(d5x) > 500:
        ax.xaxis.set_major_locator(matplotlib.dates.YearLocator())
    else:
        cov.set_date_opts(ax)
        ax.xaxis.set_major_locator(
            matplotlib.dates.MonthLocator(interval=6 if len(d5x) > 400 else 2 if len(d5x) > 100 else 1)
        )
        fig.autofmt_xdate()
    ax.grid(which="minor", alpha=0.5)
    ax.set_title(DASR_LONGSTAMP)
    ax.set_ylim(bottom=0)
    # d5x.plot(lw=0.4)
    # cov.set_date_opts(ax)


if plt_experimental:
    for bl in ("Österreich",):  # cov.SHORTNAME_BY_BUNDESLAND:
        for agefrom in range(0, 85 + 1, 5):
            plt_d5y(bl=bl, agefrom=agefrom, ageto=agefrom + 5 if agefrom != 85 else 100)

In [ ]:
def plt_more_deaths(periodname="jun_year", periodx="1ydate", periodsum="jycprate"):
    maxyear = dasr[periodname].max()
    minyear = maxyear - 10
    selbl = "Österreich"
    sid = next(i[0] for i in cov.BUNDESLAND_BY_ID.items() if i[1] == selbl)
    if sid == 10:
        sid = 0
    pdata = dasr.query(f"state_id == {sid} and sex == 'mf'").sort_values(
        by="date"
    )  # .sort_values(by="year", ascending=False, kind="stable")
    # pdata.set_index("date")["ycrate"].plot()

    plt.figure()
    is_old = pdata[periodname] <= minyear
    pdata_old = pdata[is_old].query(periodname + " >= " + repr(pdata[periodname].min()))
    pdata_new = pdata[~is_old].copy()
    if periodname.startswith("jun_"):
        pdata_new[periodname + "_l"] = (
            pdata_new[periodname].astype(str) + "/" + (pdata_new[periodname] % 100 + 1).astype(str)
        )
    else:
        pdata_new[periodname + "_l"] = pdata_new[periodname]
    # display(pdata_new)
    paln = "tab10"  # cov.un_l_cmap
    pal = collections.deque(sns.color_palette(paln, n_colors=maxyear - minyear)[::-1])
    # pal.rotate(-1)
    pal = list(pal)
    pdata_exact = pdata_new if estimate_count == 0 else pdata_new[:-estimate_count]
    drawparams = dict(x=periodx, sort=False, y=periodsum, hue=periodname + "_l", marker=".", mew=0)
    ax = sns.lineplot(pdata_exact, palette=pal, **drawparams)
    if estimate_count > 0:
        estimated = pdata_new[-estimate_count - 1 :]
        sns.lineplot(estimated, ls=":", palette=pal[-estimated[periodname].nunique() :], legend=False, **drawparams)
    # sns.color_palette("plasma", desat=.3, n_colors=pdata_old["year"].nunique()))
    cov.labelend2(ax, pdata_new, periodsum, cats=periodname, x=periodx, shorten=lambda y: str(y % 100), colorize=pal)
    if True:
        ax.set_ylim(*ax.get_ylim())
        # display(periodx, periodsum, periodname, pdata_old.dtypes)
        ax = sns.lineplot(
            pdata_old,
            x=periodx,
            sort=False,
            y=periodsum,
            zorder=1,
            hue=periodname,
            palette="Greys",
            alpha=0.5,
            lw=0.7,
            units=periodname,
            estimator=None,
            legend=False,
        )
    if "date" in periodx:
        cov.set_date_opts(ax, showyear=False)
    # cov.labelend2(ax, pdata_old, periodsum, cats=periodname, x=periodx, shorten = lambda y: format(y % 100, "02d"))
    # sns.lineplot(pdata_new, x="week", y="rate", hue="year", palette="Paired")
    ax.legend(ncol=4)
    ax.set_ylabel("Altersstandardisierte Sterberate (Promille)")
    if "date" not in periodx:
        ax.set_xlabel("Kalenderwoche" + (" (im Folgejahr weiternummeriert)" if periodx.startswith("jun_") else ""))
    else:
        ax.set_xlabel(None)
    fig = ax.figure
    fig.suptitle(
        f"{selbl}: Altersstandardisierte Sterberaten",
        # y=0.96
    )
    ax.set_title(
        "Kumulative Summe der Differenz der wöchentlichen Sterberate zum Allzeit-Wochen-Minimum von "
        + str(pdata["rate"].min())
        + "\n"
        + DASR_LONGSTAMP
    )
    if periodname.startswith("jun_"):
        ax.axvline(pd.to_datetime(date(pdata_new[periodx].dt.year.max(), 1, 1)), color="k", ls="--", zorder=1, lw=1)
    # ax.xaxis.set_major_formatter(lambda x, _: str((int(x - 1) + 25) % 53 + 1))


plt_more_deaths("year", "1ydate", "ycprate")
plt_more_deaths("jun_year", "jun_1ydate", "jycprate")

In [ ]:
import collections


def plt_weekly_deaths(periodname="jun_year", periodx="jun_1ydate", selbl="Österreich"):
    firstnew = dasr[periodname].unique()[-10]
    pdata_new = dasr.query(f"state == '{selbl}' and sex == 'mf' and {periodname} >= {firstnew}").copy()
    if periodname.startswith("jun_"):
        pdata_new[periodname + "_l"] = (
            pdata_new[periodname].astype(str) + "/" + (pdata_new[periodname] % 100 + 1).astype(str)
        )
    else:
        pdata_new[periodname + "_l"] = pdata_new[periodname]
    pdata_old = dasr.query(f"state == '{selbl}' and sex == 'mf' and {periodname} < {firstnew} and {periodname} >= 2000")
    lastyear_name = pdata_new[periodname].max()
    lastyear = pdata_new[pdata_new[periodname] == lastyear_name]
    pal = collections.deque(sns.color_palette(n_colors=pdata_new[periodname].nunique())[::-1])
    # pal.rotate(-1)
    pal = list(pal)
    ax = sns.lineplot(
        pdata_new[pdata_new[periodname] != lastyear_name],
        x=periodx,
        sort=False,
        y="rate",
        hue=periodname + "_l",
        palette=pal[:-1],
        marker=".",
        mew=0
        # size=[2] * (len(pdata_new) - len(lastyear)) + [3] * len(lastyear)
    )
    last_draw_params = dict(
        x=periodx, sort=False, y="rate", hue=periodname + "_l", palette=pal[-1:], marker=".", mew=0, lw=2
    )
    if estimate_count == 0:
        ax = sns.lineplot(pdata_new[pdata_new[periodname] == lastyear_name], markersize=10, **last_draw_params)
    else:
        ax = sns.lineplot(
            pdata_new[pdata_new[periodname] == lastyear_name].iloc[:-estimate_count],
            markersize=10,
            **last_draw_params,
        )
        ax = sns.lineplot(
            pdata_new[pdata_new[periodname] == lastyear_name].iloc[-estimate_count - 1 :],
            ls=":",
            markersize=7,
            legend=False,
            **last_draw_params,
        )
    ax.set_ylim(*ax.get_ylim())
    ax = sns.lineplot(
        pdata_old,
        x=periodx,
        sort=False,
        y="rate",
        legend=False,
        hue=periodname,
        palette="Greys",
        mew=0,
        color="k",
        alpha=0.4,
        units=periodname,
        estimator=None,
        zorder=1,
        lw=0.7,
    )
    # sns.color_palette("plasma", desat=.3, n_colors=pdata_old["year"].nunique()))
    if periodname.startswith("jun_"):
        cov.labelend2(
            ax,
            pdata_new,
            "rate",
            cats=periodname,
            x=periodx,
            shorten=lambda y: str(y % 100),
            colorize=pal,
            ends=(True, False),
        )
        lastlinedata = pdata_new[pdata_new[periodname] == pdata_new[periodname].iloc[-1]]
        over_jan = lastlinedata["year"].iloc[0] != lastlinedata["year"].iloc[-1]
        cov.labelend2(
            ax,
            pdata_new,
            "rate",
            cats=periodname,
            x=periodx,
            shorten=lambda y: str(y % 100 + 1 if over_jan or y != pdata_new[periodname].iat[-1] else y % 100),
            colorize=pal,
            ends=(False, True),
        )
    else:
        cov.labelend2(
            ax,
            pdata_new,
            "rate",
            cats=periodname,
            x=periodx,
            shorten=lambda y: str(y % 100),
            colorize=pal,
            ends=(True, True),
        )
    cov.set_date_opts(ax, showyear=False)
    ax.set_ylabel("Altersstandardisierte Sterberate (Promille)")
    # ax.set_xlabel("Kalenderwoche (im Folgejahr weiternummeriert)")
    fig = ax.figure
    fig.suptitle(f"{selbl}: Altersstandardisierte Sterberaten je Kalenderwoche", y=0.96)
    ax.set_title(DASR_LONGSTAMP + ESTIMATE_DISCLAIMER)
    ax.legend()
    ax.set_xlabel(None)
    if periodname.startswith("jun_"):
        ax.axvline(pd.to_datetime(date(pdata_new[periodx].dt.year.max(), 1, 1)), color="k", ls="--", zorder=1, lw=1)


plt_weekly_deaths()
plt.figure()
plt_weekly_deaths("year", "1ydate")

In [ ]:
def draw_x():
    pal = sns.color_palette("tab10", n_colors=9)
    periodname = "jun_year"
    periodx = "date"
    cratename = "jycprate"
    year = 2023
    pdata = dasr.query(f"{periodname} == {year} and sex == 'mf' and state_id != 0").copy()
    rate_min0 = pdata["rate"].min()
    pdata["prate"] = pdata["rate"] - rate_min0
    pdata["jycprate"] = pdata.groupby(["jun_year", "state_id", "sex"])["prate"].cumsum()
    pdata["ycprate"] = pdata.groupby(["year", "state_id", "sex"])["prate"].cumsum()
    ax = sns.lineplot(
        pdata, sort=False, x=periodx, y=cratename, hue="state", palette=pal, marker=".", mew=0
    )  # sns.color_palette("plasma", desat=.3, n_colors=pdata_old["year"].nunique()))
    cov.labelend2(ax, pdata, cratename, cats="state", x=periodx, colorize=pal, ends=(True, True))
    ax.set_ylabel("Altersstandardisierte Sterberate kumulativ (Promille)")
    # ax.set_xlabel("Kalenderwoche")
    ax.legend(ncol=2)
    fig = ax.figure
    fig.suptitle(
        f"Österreich: Altersstandardisierte Sterberaten {year}",
        # y=0.96
    )
    ax.set_title(
        "Kumulative Summe der Differenz der wöchentlichen Sterberate zum Wochen-Minimum im Ausschnitt ("
        + str(rate_min0)
        + ")\n"
        + DASR_LONGSTAMP
    )
    cov.set_date_opts(ax)
    ax.set_xlabel(None)


draw_x()

In [ ]:
def plt_dw_bl_roll3():
    pdata = dasr.query("sex == 'mf' and state_id != 0").copy().set_index(["date", "state_id"])
    rwidth = 3
    pdata["rrate"] = pdata["rate"].groupby("state_id").transform(lambda s: s.rolling(rwidth).mean())
    pdata.reset_index(inplace=True)
    pdata = pdata[pdata["date"] >= pdata.iloc[-1]["date"] - timedelta(400)]
    ax = sns.lineplot(
        pdata,
        sort=False,
        x="date",
        y="rrate",
        hue="state",
        palette="tab10",
        # marker=".", mew=0,
        alpha=0.8,
    )  # sns.color_palette("plasma", desat=.3, n_colors=pdata_old["year"].nunique()))
    cov.labelend2(ax, pdata, "rrate", cats="state", x="date", colorize=sns.color_palette(n_colors=9), ends=(True, True))
    ax.set_ylabel("Altersstandardisierte Sterberate pro Woche (Promille)")
    # ax.set_xlabel("Kalenderwoche")
    ax.legend(ncol=2, fontsize="small")
    fig = ax.figure
    fig.suptitle(f"Österreich: Altersstandardisierte Sterberaten, {rwidth}-Wochen-Schnitt (nachlaufend)", y=0.96)
    # ax.set_title("Kumulative Summe der Differenz der wöchentlichen Sterberate zum Wochen-Minimum im Ausschnitt (" + str(rate_min0) + ")")
    cov.set_date_opts(ax)
    fig.autofmt_xdate()
    # ax.xaxis.set_major_locator(matplotlib.dates.MonthLocator(interval=2))
    # ax.xaxis.set_minor_locator(matplotlib.dates.MonthLocator())
    # ax.grid(which="minor", axis="x", lw=0.3)
    ax.set_title(DASR_LONGSTAMP)
    ax.set_xlabel(None)


plt_dw_bl_roll3()

In [ ]:
bl = "Österreich"
pdata = dasr.query(f"state == '{bl}' and sex == 'mf'").set_index("date")["rate"]
rwidth = 12
start = pd.to_datetime("1999-01-01")
fig, ax = plt.subplots(layout="constrained")
# ax.plot(pdata.rolling(53).quantile(0.1).loc[start:],
#         label="10. Perzentil über 53 Wochen")
ax.plot(pdata.rolling(12).mean(), label=f"{rwidth}-Wochen-Schnitt", color="C1")  # .loc[start:],
m = ax.get_ylim()[1]
ax.plot(
    pdata, marker=".", mew=0, lw=0, label=f"Einzelwochen (Jahre 2000, 2003 oben abgeschnitten)", color="C1", alpha=0.4
)  # .loc[start:],
ax.set_ylim(top=m)
# ax.set_ylim(*ax.get_ylim())
# ax.plot(pdata.rolling(53).quantile(0.9).loc[start:],
#        label="90. Perzentil über 53 Wochen", color="C2")
# ax.plot(pdata.loc[start:], color="grey", lw=0.7, label="Altersstandardisierte Sterberate/Woche",
#       zorder=1)
fig.legend(frameon=False, ncol=2, loc="lower center", bbox_to_anchor=(0.5, 0.98))
fig.suptitle(f"{bl}: Altersstandardisierte Sterberaten", y=1.06)
ax.set_title(DASR_LONGSTAMP)
ax.set_ylabel("Altersstandardisierte Sterberate (Promille)")
ax.xaxis.set_major_locator(matplotlib.dates.YearLocator(2))
ax.xaxis.set_minor_locator(matplotlib.dates.YearLocator())
ax.grid(which="minor", alpha=0.5)
# ax.set_xlim(left=pd.to_datetime("2015-01-01"))

# Influenza und Atemwegserkrankungen

* Daten von: Agentur für Gesundheit und Ernährungssicherheit (AGES), Zentrum für Virologie der Medizinischen Universität Wien,  Diagnostisches Influenzanetzwerk Österreich (DINÖ)
* Daten-URLs:
  * <https://www.ages.at/mensch/krankheit/krankheitserreger-von-a-bis-z/grippe>
  * <https://www.virologie.meduniwien.ac.at/wissenschaft-forschung/virus-epidemiologie/influenza-projekt-diagnostisches-influenzanetzwerk-oesterreich-dinoe/>
* Daten-Aktualisierung: Wöchentlich; Daten werden nur ca. ab Kalenderwoche 40 (Anfang Oktober) bis Ende der Grippewelle oder max. ca. KW 20 (Mitte Mai) aktualisiert

In [ ]:
def onlyelem(xs):
    xs = iter(xs)
    v = next(xs)
    try:
        next(xs)
    except StopIteration:
        return v
    raise ValueError("More than one element")


grippe = onlyelem(
    pd.read_html(
        util.DATAROOT / "ages-epi-misc/grippe_latest.html",
        match="Influenza A/H3N2 Fall",
        thousands=".",
        decimal=",",
    )
)
FLU_SEASON_NAME = "2023/24"
FLU_SENTI_STAMP = f"Influenza in Österreich, Sentinel-Proben {FLU_SEASON_NAME} (AGES/DINÖ)"
grippe.rename(columns={"Week": "Woche"}, errors="ignore", inplace=True)
gfallcol = [c for c in grippe.columns if " Fall" in c]
grippe["Woche"] = grippe["Woche"].astype(str)
grippe.set_index("Woche", inplace=True, verify_integrity=True)
npos = grippe[gfallcol].sum(axis=1)
ntest = (npos / grippe["Positiv-Rate (positive Proben/Untersuchte Proben)"]).round()

In [ ]:
def plt_flu_posrate():
    ppos = grippe[gfallcol].div(ntest, axis=0)
    # display(ntest)
    # display(grippe[gfallcol])
    ppos.rename(columns={c: '% ' + c.replace(" Fall", "-positiv") for c in ppos.columns}, inplace=True)
    color = [f"C{i}" for i in range(len(ppos.columns))]
    ax = ppos.plot(color=color)
    ppos.where(lambda x: x != 0).plot(ax=ax, lw=0, marker="o", markersize=10, alpha=0.8, color=color, legend=False)
    fig = ax.figure
    ax.set_ylim(bottom=0, top=max(ax.get_ylim()[1], 0.05))
    cov.set_percent_opts(ax)
    ax.set_ylabel("Positivrate")
    fig.suptitle(FLU_SENTI_STAMP, y=0.93)
    ax.xaxis.set_major_locator(matplotlib.ticker.MaxNLocator(integer=True))


if plt_experimental:
    plt_flu_posrate()

In [ ]:
def plt_flu_posrate_and_cnt():
    ax = grippe.plot(y=gfallcol, marker="o", markersize=10, alpha=0.8)
    cov.set_logscale(ax)
    ax2 = ax.twinx()
    grippe.plot(y=[c for c in grippe.columns if "Positiv-Rate" in c], ax=ax2, color="k", marker="s", markersize=4)
    ax2.grid(False)
    ax2.get_legend().remove()
    ax.get_legend().remove()
    fig = ax.figure
    fig.suptitle(FLU_SENTI_STAMP, y=1.05)
    fig.legend(loc="upper center", ncol=2, frameon=False, bbox_to_anchor=(0.5, 1))
    ax.set_ylabel("Anzahl positiver Proben (log)")
    ax2.set_ylabel("Positivrate")
    cov.set_percent_opts(ax2)
    ax.set_ylim(bottom=0.9)
    ax2.set_ylim(bottom=0, top=max(ax2.get_ylim()[1], 0.05))
    ax.yaxis.set_major_formatter(matplotlib.ticker.StrMethodFormatter("{x:.0f}"))
    ax.xaxis.set_major_locator(matplotlib.ticker.MaxNLocator(integer=True))


plt_flu_posrate_and_cnt()

In [ ]:
def plt_ntest():
    ax = (ntest).plot(marker="o")
    ax.set_ylim(0)
    ax.yaxis.set_major_formatter(matplotlib.ticker.StrMethodFormatter("{x:.0f}"))
    ax.figure.suptitle(FLU_SEASON_NAME + ": Anzahl Sentinel-Proben (positiv+negativ) / KW (geschätzt)", y=0.93)
    ax.xaxis.set_major_locator(matplotlib.ticker.MaxNLocator(integer=True))


if plt_experimental:
    plt_ntest()

In [ ]:
def print_gh_run_info():
    run_id = os.environ.get("GITHUB_RUN_ID")
    if not run_id:
        return
    tools_repo = os.environ['GITHUB_REPOSITORY']
    tools_commit = os.environ['GITHUB_SHA']
    server_url = os.environ.get('GITHUB_SERVER_URL', 'https://github.com')
    repo_url = f"{server_url}/{tools_repo}"
    run_url = f"{repo_url}/actions/runs/{run_id}"
    data_commit = os.environ.get("COVDATA_COMMIT")
    display(
        Markdown(
            f"* GitHub Actions run: <{run_url}> (#{os.environ['GITHUB_RUN_NUMBER']}+{os.environ['GITHUB_RUN_ATTEMPT']})\n"
            f"* Generator: [{tools_repo}@{tools_commit}]({repo_url}/commit/{tools_commit}) ({os.environ.get('TOOLS_DATE', '?')})\n"
            f"* Data: [zeitferne/covidat-data@{data_commit or '?'}](https://github.com/zeitferne/covidat-data/commit/{data_commit}) ({os.environ.get('COVDATA_DATE', '?')})\n"
        )
    )


print_gh_run_info()